In [6]:
%pip install ipywidgets llama-index-core llama-index-readers-file transformers llama-index-llms-ollama llama-index-embeddings-huggingface trafilatura elasticsearch tabulate spacy tqdm llama-index-embeddings-jinaai markdown haystack-ai boilerpy3 python-slugify

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip uninstall slugify

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: slugify 0.0.1
Uninstalling slugify-0.0.1:
  Would remove:
    /home/bizon/.pyenv/versions/3.11.10/envs/.venv.3.11.10/bin/slugify
    /home/bizon/.pyenv/versions/3.11.10/envs/.venv.3.11.10/lib/python3.11/site-packages/slugify-0.0.1.dist-info/*
    /home/bizon/.pyenv/versions/3.11.10/envs/.venv.3.11.10/lib/python3.11/site-packages/slugify.py
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from llama_index.core import Document
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core.evaluation import DatasetGenerator, FaithfulnessEvaluator, RelevancyEvaluator
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
import time
import concurrent.futures
from tabulate import tabulate
import trafilatura
from tqdm import tqdm
from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.llms.ollama import Ollama
import time
import concurrent.futures
from tabulate import tabulate
import trafilatura
from unstructured.partition.html import partition_html


# Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

query = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "description"}}
                ],
                "minimum_should_match": 1,
                "should": [
                    {
                        "match_phrase": {
                        "location": "GM0141"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1896"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0180"
                        }
                    }
                ]
            }
        }
    }

# Fetch documents from Elasticsearch
results = es.search(index="jodal_documents7", body=query, size=10)

documents = []
for hit in results['hits']['hits']:
    doc_id = hit['_id']
    title = hit['_source'].get('title', '')
    description_html = hit['_source'].get('description', '')
    
    description_md = trafilatura.extract(description_html, output_format='markdown', include_links=False, include_images=False, include_formatting=False)
        
    doc = Document(text=description_md, metadata={"title": title, "id": doc_id})
    documents.append(doc)

# Define chunking strategies and sizes
chunking_strategies = {
    "trafilatura",
    # "unstructured"
}
chunk_sizes = [512, 1024]
embedding_models = {
    "HuggingFaceEmbedding": HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large"),
    "JinaEmbedding": JinaEmbedding(
        model_name="jinaai/jina-embeddings-v3", 
        api_key="jina_d3fa0f56a776410ebaf814237850e4386RC6sX7hUEKXB4nRZToG3OXkpsg1",
        task="retrieval.passage"
    )
}

# Function to save chunks to markdown files
def save_chunks(chunks, strategy, size):
    dir_path = f"data/documents/{strategy}_{size}"
    os.makedirs(dir_path, exist_ok=True)
    
    for i, chunk in enumerate(chunks):
        filename = f"{chunk.metadata['id']}-{i}_{chunk.metadata['title']}.md"
        filepath = os.path.join(dir_path, filename)
        with open(filepath, 'w') as f:
            f.write(chunk.text)

# Process documents with different chunking strategies and sizes
async def process_chunking(chunking_strategy, size, embed_model_name, embed_model):
    print(f"Processing {chunking_strategy} with chunk size {size} and embedding model {embed_model_name}")
        
    # split documents into chunks
    chunk_size = size
    all_chunks = []
    for doc in documents:
        text = doc.text
        doc_chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
        for chunk in doc_chunks:
            all_chunks.append(Document(text=chunk, metadata=doc.metadata))
    
    print(f"Generated {len(all_chunks)} chunks")
    
    save_chunks(all_chunks, chunking_strategy, size)

    llm = Ollama(model="llama3.1:8b")
    
    Settings.llm = llm
    Settings.embed_model = embed_model
    
    index = VectorStoreIndex(all_chunks)
    query_engine = index.as_query_engine()
    
    data_generator = DatasetGenerator.from_documents(all_chunks)
    print("Generating evaluation questions...")
    try:
        eval_questions = await data_generator.agenerate_questions_from_nodes(num=10)
        print(f"Generated {len(eval_questions)} questions")
    except Exception as e:
        print(f"Error generating questions: {str(e)}")
        eval_questions = []
    
    # Save questions to JSON file
    questions_file = f"data/questions_{chunking_strategy}_{size}_{embed_model_name}.json"
    with open(questions_file, 'w') as f:
        json.dump(eval_questions, f)
    
    if not eval_questions:
        print(f"Warning: No evaluation questions generated for {chunking_strategy} with chunk size {size} and embedding model {embed_model_name}")
        return {
            "Strategy": chunking_strategy,
            "Chunk Size": size,
            "Embedding Model": embed_model_name,
            "Avg Response Time": "N/A",
            "Avg Faithfulness": "N/A",
            "Avg Relevancy": "N/A"
        }
    
    faithfulness_evaluator = FaithfulnessEvaluator()
    relevancy_evaluator = RelevancyEvaluator()
    
    total_response_time = 0
    total_faithfulness = 0
    total_relevancy = 0
    
    for question in tqdm(eval_questions, desc="Evaluating questions"):
        start_time = time.time()
        response = await query_engine.aquery(question)
        total_response_time += time.time() - start_time
        
        faithfulness_result = await faithfulness_evaluator.aevaluate_response(response=response)
        relevancy_result = await relevancy_evaluator.aevaluate_response(query=question, response=response)
        
        total_faithfulness += faithfulness_result.passing
        total_relevancy += relevancy_result.passing
    
    num_questions = len(eval_questions)
    avg_response_time = total_response_time / num_questions if num_questions > 0 else 0
    avg_faithfulness = total_faithfulness / num_questions if num_questions > 0 else 0
    avg_relevancy = total_relevancy / num_questions if num_questions > 0 else 0
    
    return {
        "Strategy": chunking_strategy,
        "Chunk Size": size,
        "Embedding Model": embed_model_name,
        "Avg Response Time": f"{avg_response_time:.2f}s",
        "Avg Faithfulness": f"{avg_faithfulness:.2f}",
        "Avg Relevancy": f"{avg_relevancy:.2f}"
    }

# Use parallel processing to speed up the evaluation
results = []
for embed_model_name, embed_model in embedding_models.items():
    for chunking_strategy in chunking_strategies:
        for size in chunk_sizes:
            result = await process_chunking(chunking_strategy, size, embed_model_name, embed_model)
            results.append(result)

# Output results in a table
print(tabulate(results, headers="keys", tablefmt="grid"))



/tmp/ipykernel_71697/28177229.py:64: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  results = es.search(index="jodal_documents7", body=query, size=10)


Processing trafilatura with chunk size 512 and embedding model HuggingFaceEmbedding
Generated 10 chunks


/home/bizon/.pyenv/versions/3.11.10/envs/.venv.3.11.10/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


Generating evaluation questions...


/home/bizon/.pyenv/versions/3.11.10/envs/.venv.3.11.10/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


...

Generating evaluation questions...


/home/bizon/.pyenv/versions/3.11.10/envs/.venv.3.11.10/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


Generated 10 questions


Evaluating questions: 100%|██████████| 10/10 [01:15<00:00,  7.54s/it]

+-------------+--------------+----------------------+---------------------+--------------------+-----------------+
| Strategy    |   Chunk Size | Embedding Model      | Avg Response Time   |   Avg Faithfulness |   Avg Relevancy |
+=============+==============+======================+=====================+====================+=================+
| trafilatura |          512 | HuggingFaceEmbedding | 3.86s               |                1   |             0.9 |
+-------------+--------------+----------------------+---------------------+--------------------+-----------------+
| trafilatura |         1024 | HuggingFaceEmbedding | 3.35s               |                0.9 |             0.9 |
+-------------+--------------+----------------------+---------------------+--------------------+-----------------+
| trafilatura |          512 | JinaEmbedding        | 3.74s               |                1   |             0.7 |
+-------------+--------------+----------------------+---------------------+-----

In [8]:
async def test_ollama():
    llm = Ollama(model="llama3.1:8b")
    response = await llm.acomplete("Hello, world!")
    print("Ollama test response:", response)

await test_ollama()

Ollama test response: Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [22]:

import trafilatura
html= """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html lang="nl" xml:lang="nl" xmlns="http://www.w3.org/1999/xhtml"><head id="Head1"><meta content="text/html; charset=utf-8" http-equiv="Content-Type" /><!--
            Versienummer van conversiescript: CONVERT_VERSION_NUMBER
          --><meta content="nl-nl" http-equiv="content-language" /><meta content="text/css" http-equiv="content-style-type" /><meta content="text/javascript" http-equiv="content-script-type" /><meta content="Gemeente" name="OVERHEID.organisationType" scheme="OVERHEID.organisatietype" /><meta content="beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016" name="DC.title" /><meta content="Bergen (NH)" name="DC.creator" scheme="overheid:Gemeente" /><meta content="Bergen (NH)" name="DC.publisher" scheme="overheid:Gemeente" /><meta content="beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016" name="DCTERMS.alternative" /><meta content="college van burgemeester en wethouders" name="OVERHEID.isRatifiedBy" scheme="overheid:BestuursorgaanGemeente" /><meta content="nl" name="DC.language" scheme="DCTERMS.RFC3066" /><meta content="2016-06-16" name="DCTERMS.temporal" scheme="DCTERMS.W3CDTF" /><meta content="Participatiewet, art. 35;" name="DC.source" /><meta content="2016-05-10" name="DCTERMS.issued" scheme="DCTERMS.W3CDTF" /><meta content="Gemeenteblad, 15 juni 2016, nr. 72683;" name="DCTERMS.relation.isPartOf" scheme="OVERHEID.relation.isPartOf.publicatieblad" /><meta content="maatschappelijke zorg en welzijn" name="DC.subject" /><meta content="maatschappelijke zorg en welzijn" name="OVERHEID.category" scheme="OVERHEID.modelverordeningenDomein" /><meta content="maatschappelijke zorg en welzijn" name="DC.subject" /><meta content="maatschappelijke zorg en welzijn" name="OVERHEID.category" scheme="OVERHEID.modelverordeningenDomein" /><meta content="http://cvdr.overheid.nl" name="DCTERMS.isPartOf" scheme="DCTERMS.URI" /><meta content="regeling" name="DC.type" scheme="OVERHEID.informatietype" /><meta content="text/html" name="DC.format" scheme="IMT" /><meta content="De tekst in dit document is vrij van auteursrecht en databankenrecht" name="OVERHEID.copyright" /><meta content="CVDR407564_1" name="DC.identifier" /><title>beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016</title><link href="http://purl.org/dc/elements/1.1/" rel="schema.DC" /><link href="http://purl.org/dc/terms/" rel="schema.DCTERMS" /><link href="/cvdr/Static/Stylesheets/publicaties/cvdr.css" rel="stylesheet" type="text/css" /><link href="/cvdr/Static/Stylesheets/framework.css" rel="stylesheet" type="text/css" /><link href="/cvdr/Static/Stylesheets/default-publicaties.css" rel="stylesheet" type="text/css" /><!--[if IE 6]><link rel="stylesheet" href="/{$cssfolder}Static/Stylesheets/ie6-publicaties-correcties.css" type="text/css" /><![endif]--><!--[if IE 7]><link rel="stylesheet" href="/{$cssfolder}Static/Stylesheets/ie7-publicaties-correcties.css" type="text/css" /><![endif]--><!--[if IE 8]><link rel="stylesheet" href="/{$cssfolder}Static/Stylesheets/ie8-publicaties-correcties.css" type="text/css" /><![endif]--><link href="/cvdr/Static/Stylesheets/publicaties/op.css" rel="stylesheet" type="text/css" /><link href="/cvdr/Static/Stylesheets/publicaties/op-list.css" rel="stylesheet" type="text/css" /><link href="/cvdr/Static/Stylesheets/publicaties/op-table.css" rel="stylesheet" type="text/css" /><link href="/cvdr/Static/Stylesheets/publicaties/op-noten.css" rel="stylesheet" type="text/css" /><!--[if gte IE 6]><link href="/Static/Stylesheets/publicaties/op-iehacks.css" rel="stylesheet" type="text/css" /><![endif]--><!--[if lt IE 7]><link href="/Static/Stylesheets/publicaties/op-oldiehacks.css" rel="stylesheet" type="text/css" /><![endif]--></head><body><div class="zoekdienstlink"><a href="http://zoekdienst.overheid.nl/Zoekdienst/html.onl.nl/collectie/dcr?searchform=advanced&amp;view=advanced&amp;kb_reset_basket=true" title="Ga naar het zoekformulier voor lokale regelingen op overheid.nl.">Zoek regelingen op overheid.nl</a></div><div id="PaginaContainer"><div class="inhoud"><div id="inhoud"><div id="cvdr_meta"><div class="cvdr_header"><div class="emailmelding">
                Ziet u een fout in deze regeling?
                <a href="mailto:regelgeving@overheid.nl" id="maillink" title="Meld het ons!">Meld het ons op regelgeving@overheid.nl!</a></div><div class="headertekst">Bergen (NH)</div></div><h1 class="cvdr_meta_titel">beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016</h1><h3 class="cvdr_meta_hoofdkop">Wetstechnische informatie</h3><table><caption>Gegevens van de regeling</caption><tbody><tr><th class="meta_omschrijving" scope="row">Organisatie</th><td>Bergen (NH)</td></tr><tr><th class="meta_omschrijving" scope="row">Organisatietype</th><td>Gemeente</td></tr><tr><th class="meta_omschrijving" scope="row">Officiële naam regeling</th><td>beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016</td></tr><tr><th class="meta_omschrijving" scope="row">Citeertitel</th><td>beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016</td></tr><tr><th class="meta_omschrijving" scope="row">Vastgesteld door</th><td>college van burgemeester en wethouders</td></tr><tr><th class="meta_omschrijving" scope="row">Onderwerp</th><td>maatschappelijke zorg en welzijn</td></tr><tr><th class="meta_omschrijving" scope="row">Eigen onderwerp</th><td></td></tr></tbody></table><h4 class="cvdr_meta_kop">Opmerkingen met betrekking tot de regeling</h4><p>Geen</p><h4 class="cvdr_meta_kop">Wettelijke grondslag(en) of bevoegdheid waarop de regeling is gebaseerd</h4><p>Participatiewet, art. 35</p><h4 class="cvdr_meta_kop">Regelgeving die op deze regeling is gebaseerd (gedelegeerde regelgeving)</h4><p>Geen</p><table id="historie"><caption>Overzicht van in de tekst verwerkte wijzigingen</caption><tbody><tr><th scope="col"><p>Datum inwerkingtreding</p></th><th scope="col"><p>Terugwerkende kracht tot en met</p></th><th scope="col"><p>Datum uitwerkingtreding</p></th><th class="width20p" scope="col"><p>Betreft</p></th><th scope="col"><p>Datum ondertekening</p><p>Bron bekendmaking</p></th><th scope="col"><p>Kenmerk voorstel</p></th></tr><tr><td>16-06-2016</td><td>01-01-2016</td><td></td><td>nieuwe regeling</td><td><p>10-05-2016</p><p>Gemeenteblad, 15 juni 2016, nr. 72683</p></td><td>Onbekend</td></tr></tbody></table></div><h3 class="regelingkop">Tekst van de regeling</h3><div id="broodtekst"><div class="officiele-publicatie" lang="nl" xml:lang="nl"><div class="intitule"><h3 class="body_kop">Intitulé</h3>beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016</div><div class="_p_regeling"><a name="id1-3-2"><!----></a><div class="aanhef _p_aanhef"><a name="id1-3-2-1"><!----></a><div class="_p_preambule"><a name="id1-3-2-1-1"><!----></a><p class="al _p_al"><a name="id1-3-2-1-1-1"><!----></a>Het college van de gemeente Bergen</p><p class="al _p_al"><a name="id1-3-2-1-1-2"><!----></a>Gelet op artikel 35 van de Participatiewet;</p><p class="al _p_al"><a name="id1-3-2-1-1-3"><!----></a>Besluit vast te stellen de beleidsregels ‘meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016’</p></div></div><div class="_p_regeling-tekst"><a name="id1-3-2-2"><!----></a><div class="_p_artikel"><a name="id1-3-2-2-1"><!----></a><a name="id1-3-2-2-1-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-1-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">1</span> Begripsbepalingen</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-2-2"><!----></a>Alle begrippen die in deze beleidsregel worden gebruikt en die niet nader worden omschreven hebben dezelfde betekenis als in de Participatiewet, de Algemene wet bestuursrecht (Awb), de Gemeentewet, de Algemene wet inkomensafhankelijke regelingen (Awir) en de eerder vastgestelde beleidsregels bijzondere bijstand.</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-2"><!----></a>In deze beleidsregel wordt verstaan onder:</p><ul class="_explicit whitespace-small _p_lijst"><li class="li _p_li"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-1-2"><!----></a>het college: het college van burgemeester en wethouders;</p></div></li><li class="li _p_li"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-2-2"><!----></a>de wet: de Participatiewet;</p></div></li><li class="li _p_li"><span>c.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-3-2"><!----></a>bijzondere bijstand: de bijstand bedoeld in artikel 35 Participatiewet;</p></div></li><li class="li _p_li"><span>d.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-4-2"><!----></a>meerkosten: het gaat bij meerkosten niet om alle kosten, maar om de extra kosten die boven de gewone kosten uitkomen in relatie tot de chronische aandoening. Het betreft niet alleen directe ziekte- of zorgkosten, maar ook indirecte kosten die een gevolg kunnen zijn en samenhangen met de chronische aandoening. Als uitgangspunt voor gewone kosten kunnen de normen van het Nationaal Instituut voor Budgetvoorlichting (Nibud) gelden;</p></div></li><li class="li _p_li"><span>e.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-5-2"><!----></a>draagkracht: het gedeelte van het inkomen of vermogen dat de inwoner geacht wordt aan te wenden om in de bijzondere kosten te voorzien;</p></div></li><li class="li _p_li"><span>f.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-6-2"><!----></a>draagkrachtperiode: de periode waarover de financiële draagkracht van de inwoner wordt vastgesteld;</p></div></li><li class="li _p_li"><span>g.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-7-2"><!----></a>inkomen: het inkomen zoals bedoeld in de artikel 31 tot en met 33 Participatiewet;</p></div></li><li class="li _p_li"><span>h.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-8-2"><!----></a>minimum inkomen: een inkomen van ten hoogste 120% van de voor de inwoner op datum aanvraag geldende bijstandsnorm;</p></div></li><li class="li _p_li"><span>i.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-9-2"><!----></a>vermogen: het vermogen als bedoeld in artikel 34 lid 1 van de Participatiewet;</p></div></li><li class="li _p_li"><span>j.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-10-2"><!----></a>bescheiden vermogen: een vermogen van maximaal het in artikel 34 lid 3 van de Participatiewet genoemde bedrag; </p></div></li><li class="li _p_li"><span>k.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-11-2"><!----></a>voorliggende voorziening: de voorziening als bedoeld in artikel 15 Participatiewet;</p></div></li><li class="li _p_li"><span>l.</span><div><p class="al _p_al"><a name="id1-3-2-2-1-3-3-12-2"><!----></a>woning: een woning zoals bedoeld in artikel 1 onderdeel j Wet op de huurtoeslag, alsmede een woonwagen of woonschip, zoals bedoeld in artikel 3 lid 6 Participatiewet.</p></div></li></ul></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-2"><!----></a><a name="id1-3-2-2-2-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-2-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">2</span> Doelstelling</p><p class="al _p_al"><a name="id1-3-2-2-2-2"><!----></a>Deze beleidsregels regelen bijzondere bijstand ten behoeve van chronisch zieken en/of mensen met een chronische beperking met een minimum inkomen, zodat de meerkosten genoemd onder artikel 9 en artikel 10 geen belemmering vormen om zelfredzaam te zijn en maatschappelijk te participeren. </p></div><div class="_p_artikel"><a name="id1-3-2-2-3"><!----></a><a name="id1-3-2-2-3-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-3-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">3</span> Definitie chronische ziekte en/of chronische beperking</p><p class="al _p_al"><a name="id1-3-2-2-3-2"><!----></a>We spreken van een inwoner met een chronische ziekte of chronische beperking als die inwoner:</p><ul class="_explicit whitespace-small _p_lijst"><li class="li _p_li"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-1-2"><!----></a>het verplicht eigen risico van de zorgverzekering twee jaar achtereenvolgens voorafgaand aan de aanvraag volledig heeft opgemaakt; of</p></div></li><li class="li _p_li"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-2-2"><!----></a>het eigen risico van de zorgverzekering van het jaar voorafgaand aan de aanvraag en het lopende jaar volledig heeft opgemaakt; of</p></div></li><li class="li _p_li"><span>c.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-3-2"><!----></a>beschikt over een gehandicaptenparkeerkaart; of</p></div></li><li class="li _p_li"><span>d.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-4-2"><!----></a>een indicatie heeft voor de Wet langdurige zorg voor een periode langer dan 6 maanden; of</p></div></li><li class="li _p_li"><span>e.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-5-2"><!----></a>een indicatie heeft voor de Wet maatschappelijke ondersteuning voor een periode langer dan 6 maanden; of</p></div></li><li class="li _p_li"><span>f.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-6-2"><!----></a>op grond van de Participatiewet of Wajong ontheffing heeft gekregen van de arbeidsverplichting om medische redenen voor een periode langer dan 6 maanden; of</p></div></li><li class="li _p_li"><span>g.</span><div><p class="al _p_al"><a name="id1-3-2-2-3-3-7-2"><!----></a>een volledige WAO, WIA of Wajong uitkering heeft.</p></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-4"><!----></a><a name="id1-3-2-2-4-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-4-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">4</span> Kostendelersnorm</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-4-2-2"><!----></a>De kostendelersnorm als bedoeld in artikel 22a van de wet is van toepassing op deze beleidsregels.</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-4-3-2"><!----></a>De beleidsregels kostendelersnorm en verlagingen 2015 gemeente Bergen zijn van toepassing op aanvragen bijzondere bijstand. </p></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-5"><!----></a><a name="id1-3-2-2-5-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-5-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">5</span> Voorliggende voorzieningen</p><p class="al _p_al"><a name="id1-3-2-2-5-2"><!----></a>Indien de inwoner gebruik kan maken van een andere voorziening die voorziet in de meerkosten zoals benoemd in artikel 9 en artikel 10, dan wordt deze voorziening als voorliggende voorziening gezien. Er kan dan geen aanspraak worden gemaakt op deze regeling</p><ul class="_explicit whitespace-small _p_lijst"><li class="li _p_li"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-2"><!----></a>Als voorliggende voorziening als bedoeld in artikel 15 van de wet worden onder meer aangemerkt:</p><ul class="_explicit whitespace-small _p_lijst"><li class="li _p_li"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-3-1-2"><!----></a>de aanvullende verzekering van de Collectieve Zorgverzekering Minima (CZM);</p></div></li><li class="li _p_li"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-3-2-2"><!----></a>de Wet maatschappelijke ondersteuning 2015 (Wmo 2015); </p></div></li><li class="li _p_li"><span>c.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-3-3-2"><!----></a>de Wet langdurige zorg (Wlz);</p></div></li><li class="li _p_li"><span>d.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-3-4-2"><!----></a>de Zorgverzekeringswet (Zvw); </p></div></li><li class="li _p_li"><span>e.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-3-5-2"><!----></a>de Algemene kinderbijslagwet (Akw);</p></div></li><li class="li _p_li"><span>f.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-1-3-6-2"><!----></a>Basisverzekerering Zorg</p></div></li></ul></div></li><li class="li _p_li"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-5-3-2-2"><!----></a>De in het eerste lid genoemde voorliggende voorzieningen betreffen geen limitatieve opsomming.</p></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-6"><!----></a><a name="id1-3-2-2-6-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-6-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">6</span> De wijze en het tijdstip van aanvragen</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-6-2-2"><!----></a>Een aanvraag wordt met het daarvoor bestemde aanvraagformulier ingediend bij het college. Op het aanvraagformulier staat vermeld welke bewijsstukken belanghebbende bij de aanvraag moet overleggen.</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-6-3-2"><!----></a>een aanvraag dient te worden ingediend:</p><ul class="_explicit whitespace-small _p_lijst"><li class="li _p_li"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-6-3-3-1-2"><!----></a>voor belanghebbende die een beroep doet op artikel 9 van deze beleidsregels binnen de daarvoor geldende termijnen zoals benoemd in het vastgestelde beleid bijzondere bijstand.</p></div></li><li class="li _p_li"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-6-3-3-2-2"><!----></a>voor belanghebbende die een beroep doet op artikel 10 van deze beleidsregels binnen 4 weken na ontvangst van de specificatie waaruit blijkt hoe hoog het eigen aandeel is. Dit met uitzondering van de maanden januari 2016 tot en met mei 2016. Deze mogen met terugwerkende kracht in een keer worden ingediend in juni 2016.</p></div></li></ul></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-7"><!----></a><a name="id1-3-2-2-7-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-7-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">7</span> Draagkracht</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-2-2"><!----></a>Bij de berekening van de draagkracht wordt uitgegaan van hetgeen van de beleidsregels bijzondere bijstand 2015 gemeente Bergen, vastgesteld op 10 september 2015.</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-3-2"><!----></a>Bij de berekening van de draagkracht wordt in ieder geval rekening gehouden met de volgende aftrekbare kosten, voor zover dit niet in het eigen beleid zoals genoemd onder lid 1 is geregeld.</p><ul class="_explicit whitespace-small _p_lijst"><li class="li _p_li"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-3-3-1-2"><!----></a>eigen bijdrage CAK;</p></div></li><li class="li _p_li"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-3-3-2-2"><!----></a>eigen aandeel schoonmaakondersteuning;</p></div></li><li class="li _p_li"><span>c.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-3-3-3-2"><!----></a>kosten die worden gemaakt vanwege het niet kunnen deelnemen aan de CZM;</p></div></li><li class="li _p_li"><span>d.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-3-3-4-2"><!----></a>kosten die worden gemaakt door het niet kunnen krijgen van kwijtscheldingen gemeentelijke belastingen en waterschapsbelastingen;</p></div></li><li class="li _p_li"><span>e.</span><div><p class="al _p_al"><a name="id1-3-2-2-7-3-3-5-2"><!----></a>kosten die worden gemaakt door het niet kunnen krijgen van huur- en zorgtoeslag.</p></div></li></ul></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-8"><!----></a><a name="id1-3-2-2-8-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-8-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">8</span> Doelgroep</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-2-2"><!----></a>Voor bijzondere bijstand op grond van deze regeling komt in aanmerking de inwoner die:</p></div></li><li class="_p_lid"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-3-2"><!----></a>ingeschreven staat in de gemeente Bergen op datum van de aanvraag; en</p></div></li><li class="_p_lid"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-4-2"><!----></a>op de datum van aanvraag een inkomen heeft dat niet hoger is dan 120% van de van toepassing zijnde bijstandsnorm (dit betreft geen harde grens vanwege de aanvullingen van de draagkrachtberekening in artikel 7. Inwoners die boven de grens van het sociaal minimum leven kunnen vanwege meerkosten en het mislopen van deelname aan bepaalde regelingen dusdanige kosten maken dat zij onder de 120% uitkomen (oprekken draagkracht). Hierdoor kunnen ook zij een risico gaan lopen in zelfredzaamheid en participatie); en</p></div></li><li class="_p_lid"><span>c.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-5-2"><!----></a>de inwoner die op de datum van aanvraag een vermogen heeft dat niet hoger is dan artikel 34 van de Participatiewet; en</p></div></li><li class="_p_lid"><span>d.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-6-2"><!----></a>de inwoner die chronisch ziek is of een chronische beperking heeft zoals omschreven in artikel 3 van de beleidsregels; en</p></div></li><li class="_p_lid"><span>e.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-7-2"><!----></a>de inwoner die de Nederlandse nationaliteit heeft of daaraan gelijkgesteld</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-8-8-2"><!----></a>Indien de inwoner in een andere gemeente al een vergoeding heeft verkregen voor de meerkosten als gevolg van de chronische ziekte of chronische beperking dan heeft de inwoner geen recht op een financiële vergoeding in de huidige gemeente.</p></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-9"><!----></a><a name="id1-3-2-2-9-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-9-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">9</span> (Para)medische kosten voor chronisch zieken en chronische beperking conform deze beleidsregels</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-2-2"><!----></a>Voor bijzondere bijstand komen de volgende (para)medische noodzakelijke kosten in aanmerking, indien er geen voorliggende voorziening is:</p></div></li><li class="_p_lid"><span>a.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-3-2"><!----></a>de meerkosten van een medisch noodzakelijk dieet of aangepast voedsel;</p></div></li><li class="_p_lid"><span>b.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-4-2"><!----></a>de ten gevolge van langdurige ziekte of beperking ontstane meerkosten van slijtage van kleding en/of beddengoed;</p></div></li><li class="_p_lid"><span>c.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-5-2"><!----></a>de ten gevolge van langdurige ziekte of beperking ontstane meerkosten van bewassing van kleding en/of beddengoed;</p></div></li><li class="_p_lid"><span>d.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-6-2"><!----></a>de meerkosten van verwarming ten gevolge van een beperking of langdurige ziekte;</p></div></li><li class="_p_lid"><span>e.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-7-2"><!----></a>zelfzorggeneesmiddelen;</p></div></li><li class="_p_lid"><span>f.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-8-2"><!----></a>pedicurekosten;</p></div></li><li class="_p_lid"><span>g.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-9-2"><!----></a>vervoerskosten die buiten de regiotaxi vallen, maar wel (tijdelijk) noodzakelijk zijn;</p></div></li><li class="_p_lid"><span>h.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-10-2"><!----></a>aanvullende verzekeringspremie voor chronisch zieken, die om gegronde redenen niet over (kunnen) stappen naar de CZM/gemeentepolis.</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-9-11-2"><!----></a>De in het eerste lid genoemde kosten betreffen geen limitatieve opsomming.</p></div></li></ul></div><div class="_p_artikel"><a name="id1-3-2-2-10"><!----></a><a name="id1-3-2-2-10-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-10-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">10</span> Eigen aandeel schoonmaakondersteuning 2016</p><p class="al _p_al"><a name="id1-3-2-2-10-2"><!----></a>De inwoner die gebruik maakt van de algemene voorziening schoonmaakondersteuning die voldoet aan de criteria in artikel 7 draagkracht komt in aanmerking voor bijzondere bijstand voor de volledige kosten van het eigen aandeel voor maximaal 8 uren per vier weken. </p></div><div class="_p_artikel"><a name="id1-3-2-2-11"><!----></a><a name="id1-3-2-2-11-1"><!----></a><p class="artikel_kop _p_artikel_kop_titel"><a name="id1-3-2-2-11-1-3"><!----></a><span class="artikel_kop _p_artikel_kop_label">Artikel</span> <span class="artikel_kop _p_artikel_kop_nr">11</span> Inwerkingtreding en citeertitel</p><ul class="_explicit"><li class="_p_lid"><span>1.</span><div><p class="al _p_al"><a name="id1-3-2-2-11-2-2"><!----></a>Deze beleidsregels treden met terugwerkende kracht in werking vanaf 1 januari 2016. Dit met uitzondering van artikel 9.</p></div></li><li class="_p_lid"><span>2.</span><div><p class="al _p_al"><a name="id1-3-2-2-11-3-2"><!----></a>Artikel 9 van deze beleidsregels treedt in werking op de dag na bekendmaking tot een jaar daarna.</p></div></li><li class="_p_lid"><span>3.</span><div><p class="al _p_al"><a name="id1-3-2-2-11-4-2"><!----></a>Deze beleidsregels kunnen worden aangehaald als: beleidsregels ‘meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016’.</p></div></li></ul></div></div></div></div></div></div></div></div></body></html>"""
                
markdown = trafilatura.extract(html, output_format="markdown", favor_precision=True)
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md

elements = partition_md(text=markdown)
chunks = chunk_by_title(elements)

# Print the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}:\n{chunk}\n\n")


Chunk 0:
Artikel 1 Begripsbepalingen

1. Alle begrippen die in deze beleidsregel worden gebruikt en die niet nader worden omschreven hebben dezelfde betekenis als in de Participatiewet, de Algemene wet bestuursrecht (Awb), de Gemeentewet, de Algemene wet inkomensafhankelijke regelingen (Awir) en de eerder vastgestelde beleidsregels bijzondere bijstand.

2. In deze beleidsregel wordt verstaan onder:

a. het college: het college van burgemeester en wethouders;

b. de wet: de Participatiewet;


Chunk 1:
c. bijzondere bijstand: de bijstand bedoeld in artikel 35 Participatiewet;


Chunk 2:
d. meerkosten: het gaat bij meerkosten niet om alle kosten, maar om de extra kosten die boven de gewone kosten uitkomen in relatie tot de chronische aandoening. Het betreft niet alleen directe ziekte- of zorgkosten, maar ook indirecte kosten die een gevolg kunnen zijn en samenhangen met de chronische aandoening. Als uitgangspunt voor gewone kosten kunnen de normen van het Nationaal Instituut voor Budgetvo

In [2]:
html = "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Strict//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd\"><html lang=\"nl\" xml:lang=\"nl\" xmlns=\"http://www.w3.org/1999/xhtml\"><head id=\"Head1\"><meta content=\"text/html; charset=utf-8\" http-equiv=\"Content-Type\" /><!--\r\n            Versienummer van conversiescript: CONVERT_VERSION_NUMBER\r\n          --><meta content=\"nl-nl\" http-equiv=\"content-language\" /><meta content=\"text/css\" http-equiv=\"content-style-type\" /><meta content=\"text/javascript\" http-equiv=\"content-script-type\" /><meta content=\"Gemeente\" name=\"OVERHEID.organisationType\" scheme=\"OVERHEID.organisatietype\" /><meta content=\"Beleidsregels Beschermd Wonen Montferland 2016\" name=\"DC.title\" /><meta content=\"Montferland\" name=\"DC.creator\" scheme=\"overheid:Gemeente\" /><meta content=\"Montferland\" name=\"DC.publisher\" scheme=\"overheid:Gemeente\" /><meta content=\"Beleidsregels Beschermd Wonen Montferland 2016\" name=\"DCTERMS.alternative\" /><meta content=\"college van burgemeester en wethouders\" name=\"OVERHEID.isRatifiedBy\" scheme=\"overheid:BestuursorgaanGemeente\" /><meta content=\"nl\" name=\"DC.language\" scheme=\"DCTERMS.RFC3066\" /><meta content=\"2016-01-01/2017-01-01\" name=\"DCTERMS.temporal\" scheme=\"DCTERMS.W3CDTF\" /><meta content=\"Verordening maatschappelijke ondersteuning Montferland 2015;1.1:CVDR362927_1\" name=\"DC.source\" /><meta content=\"Algemene wet bestuursrecht, art. 4:81;1.0:v:BWBR0005537&amp;artikel=4:81\" name=\"DC.source\" /><meta content=\"2016-05-10\" name=\"DCTERMS.issued\" scheme=\"DCTERMS.W3CDTF\" /><meta content=\"gmb-2016-98225;https://zoek.officielebekendmakingen.nl/gmb-2016-98225.html\" name=\"DCTERMS.relation.isPartOf\" scheme=\"OVERHEID.relation.isPartOf.publicatieblad\" /><meta content=\"maatschappelijke zorg en welzijn\" name=\"DC.subject\" /><meta content=\"maatschappelijke zorg en welzijn\" name=\"OVERHEID.category\" scheme=\"OVERHEID.modelverordeningenDomein\" /><meta content=\"maatschappelijke zorg en welzijn\" name=\"DC.subject\" /><meta content=\"maatschappelijke zorg en welzijn\" name=\"OVERHEID.category\" scheme=\"OVERHEID.modelverordeningenDomein\" /><meta content=\"http://cvdr.overheid.nl\" name=\"DCTERMS.isPartOf\" scheme=\"DCTERMS.URI\" /><meta content=\"regeling\" name=\"DC.type\" scheme=\"OVERHEID.informatietype\" /><meta content=\"text/html\" name=\"DC.format\" scheme=\"IMT\" /><meta content=\"De tekst in dit document is vrij van auteursrecht en databankenrecht\" name=\"OVERHEID.copyright\" /><meta content=\"CVDR411730_1\" name=\"DC.identifier\" /><title>Beleidsregels Beschermd Wonen Montferland 2016</title><link href=\"http://purl.org/dc/elements/1.1/\" rel=\"schema.DC\" /><link href=\"http://purl.org/dc/terms/\" rel=\"schema.DCTERMS\" /><link href=\"/cvdr/Static/Stylesheets/publicaties/cvdr.css\" rel=\"stylesheet\" type=\"text/css\" /><link href=\"/cvdr/Static/Stylesheets/framework.css\" rel=\"stylesheet\" type=\"text/css\" /><link href=\"/cvdr/Static/Stylesheets/default-publicaties.css\" rel=\"stylesheet\" type=\"text/css\" /><!--[if IE 6]><link rel=\"stylesheet\" href=\"/{$cssfolder}Static/Stylesheets/ie6-publicaties-correcties.css\" type=\"text/css\" /><![endif]--><!--[if IE 7]><link rel=\"stylesheet\" href=\"/{$cssfolder}Static/Stylesheets/ie7-publicaties-correcties.css\" type=\"text/css\" /><![endif]--><!--[if IE 8]><link rel=\"stylesheet\" href=\"/{$cssfolder}Static/Stylesheets/ie8-publicaties-correcties.css\" type=\"text/css\" /><![endif]--><link href=\"/cvdr/Static/Stylesheets/publicaties/op.css\" rel=\"stylesheet\" type=\"text/css\" /><link href=\"/cvdr/Static/Stylesheets/publicaties/op-list.css\" rel=\"stylesheet\" type=\"text/css\" /><link href=\"/cvdr/Static/Stylesheets/publicaties/op-table.css\" rel=\"stylesheet\" type=\"text/css\" /><link href=\"/cvdr/Static/Stylesheets/publicaties/op-noten.css\" rel=\"stylesheet\" type=\"text/css\" /><!--[if gte IE 6]><link href=\"/Static/Stylesheets/publicaties/op-iehacks.css\" rel=\"stylesheet\" type=\"text/css\" /><![endif]--><!--[if lt IE 7]><link href=\"/Static/Stylesheets/publicaties/op-oldiehacks.css\" rel=\"stylesheet\" type=\"text/css\" /><![endif]--></head><body><div class=\"zoekdienstlink\"><a href=\"http://zoekdienst.overheid.nl/Zoekdienst/html.onl.nl/collectie/dcr?searchform=advanced&amp;view=advanced&amp;kb_reset_basket=true\" title=\"Ga naar het zoekformulier voor lokale regelingen op overheid.nl.\">Zoek regelingen op overheid.nl</a></div><div id=\"PaginaContainer\"><div class=\"inhoud\"><div id=\"inhoud\"><div id=\"cvdr_meta\"><div class=\"cvdr_header\"><div class=\"emailmelding\">\r\n                Ziet u een fout in deze regeling?\r\n                <a href=\"mailto:regelgeving@overheid.nl\" id=\"maillink\" title=\"Meld het ons!\">Meld het ons op regelgeving@overheid.nl!</a></div><div class=\"headertekst\">Montferland</div></div><h1 class=\"cvdr_meta_titel\">Beleidsregels Beschermd Wonen Montferland 2016</h1><h3 class=\"cvdr_meta_hoofdkop\">Wetstechnische informatie</h3><table><caption>Gegevens van de regeling</caption><tbody><tr><th class=\"meta_omschrijving\" scope=\"row\">Organisatie</th><td>Montferland</td></tr><tr><th class=\"meta_omschrijving\" scope=\"row\">Organisatietype</th><td>Gemeente</td></tr><tr><th class=\"meta_omschrijving\" scope=\"row\">Officiële naam regeling</th><td>Beleidsregels Beschermd Wonen Montferland 2016</td></tr><tr><th class=\"meta_omschrijving\" scope=\"row\">Citeertitel</th><td>Beleidsregels Beschermd Wonen Montferland 2016</td></tr><tr><th class=\"meta_omschrijving\" scope=\"row\">Vastgesteld door</th><td>college van burgemeester en wethouders</td></tr><tr><th class=\"meta_omschrijving\" scope=\"row\">Onderwerp</th><td>maatschappelijke zorg en welzijn</td></tr><tr><th class=\"meta_omschrijving\" scope=\"row\">Eigen onderwerp</th><td></td></tr></tbody></table><h4 class=\"cvdr_meta_kop\">Opmerkingen met betrekking tot de regeling</h4><p>Geen.</p><h4 class=\"cvdr_meta_kop\">Wettelijke grondslag(en) of bevoegdheid waarop de regeling is gebaseerd</h4><ol class=\"grondslag\"><li class=\"li\"><a href=\"http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/362927/362927_1.html\">Verordening maatschappelijke ondersteuning Montferland 2015</a></li><li class=\"li\"><a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0005537/article=4:81\">Algemene wet bestuursrecht, art. 4:81</a></li></ol><h4 class=\"cvdr_meta_kop\">Regelgeving die op deze regeling is gebaseerd (gedelegeerde regelgeving)</h4><p>Geen.</p><table id=\"historie\"><caption>Overzicht van in de tekst verwerkte wijzigingen</caption><tbody><tr><th scope=\"col\"><p>Datum inwerkingtreding</p></th><th scope=\"col\"><p>Terugwerkende kracht tot en met</p></th><th scope=\"col\"><p>Datum uitwerkingtreding</p></th><th class=\"width20p\" scope=\"col\"><p>Betreft</p></th><th scope=\"col\"><p>Datum ondertekening</p><p>Bron bekendmaking</p></th><th scope=\"col\"><p>Kenmerk voorstel</p></th></tr><tr><td>01-01-2016</td><td></td><td>01-01-2017</td><td>Nieuwe regeling</td><td><p>10-05-2016</p><p><a href=\"https://zoek.officielebekendmakingen.nl/gmb-2016-98225.html\">gmb-2016-98225</a></p></td><td>16int01171</td></tr></tbody></table></div><h3 class=\"regelingkop\">Tekst van de regeling</h3><div id=\"broodtekst\"><div class=\"officiele-publicatie\" lang=\"nl\" xml:lang=\"nl\"><div class=\"intitule\"><h3 class=\"body_kop\">Intitulé</h3>Beleidsregels Beschermd Wonen Gemeente Montferland 2016</div><div class=\"_p_regeling\"><a name=\"id1-3-2\"><!----></a><div class=\"aanhef _p_aanhef\"><a name=\"id1-3-2-1\"><!----></a><div class=\"_p_preambule\"><a name=\"id1-3-2-1-1\"><!----></a><p class=\"al _p_al\"><a name=\"id1-3-2-1-1-1\"><!----></a>Het college van burgemeester en wethouders van Montferland</p><p class=\"al _p_al\"><a name=\"id1-3-2-1-1-2\"><!----></a>Overwegende dat het de bevoegdheid heeft om regels te stellen omtrent de te verstrekken maatwerkvoorzieningen in de vorm van zorg in natura en in de vorm van het persoonsgebonden budget in het kader van de <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035362\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035362\">Wet maatschappelijke ondersteuning 2015</a>;</p><p class=\"al _p_al\"><a name=\"id1-3-2-1-1-3\"><!----></a>Gelet op de <a href=\"http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/362927/362927_1.html\" title=\"link naar publicatie http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/362927/362927_1.html\">Verordening maatschappelijke ondersteuning Montferland 2015</a>, verder te noemen Verordening, en <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0005537/article=4:81\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0005537/article=4:81\">artikel 4:81 Algemene wet bestuursrecht</a>;</p><p class=\"al _p_al\"><a name=\"id1-3-2-1-1-4\"><!----></a>Besluit:</p><p class=\"al _p_al\"><a name=\"id1-3-2-1-1-5\"><!----></a>Vast te stellen het navolgende:</p><p class=\"al _p_al\"><a name=\"id1-3-2-1-1-6\"><!----></a><span class=\"vet _p_nadrukvet\">Beleidsregels Beschermd Wonen Montferland 2016 inclusief de bijbehorende toelichting</span></p></div></div><div class=\"_p_regeling-tekst\"><a name=\"id1-3-2-2\"><!----></a><div class=\"_p_artikel\"><a name=\"id1-3-2-2-1\"><!----></a><a name=\"id1-3-2-2-1-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-1-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">1</span> Begripsbepalingen</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-2\"><!----></a>Alle definities die in deze beleidsregels worden gebruikt, hebben dezelfde betekenis als in de <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035362\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035362\">Wet maatschappelijke ondersteuning 2015</a>, de hierop gebaseerde algemene maatregelen van bestuur, uitvoeringsbesluiten, de <a href=\"http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/369856/369856_1.html\" title=\"link naar publicatie http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/369856/369856_1.html\">Gewijzigde Verordening maatschappelijke ondersteuning Montferland 2015</a> en het Besluit maatschappelijke ondersteuning Montferland 2015.</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-3\"><!----></a><span class=\"italic _p_nadrukcur\">Achterhoekse gemeenten:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-4\"><!----></a>Aalten, Berkelland, Bronckhorst, Doetinchem, Montferland, Oost Gelre, Oude IJsselstreek, Winterswijk.</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-5\"><!----></a><span class=\"italic _p_nadrukcur\">Beschermd Wonen:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-6\"><!----></a>Bij Beschermd Wonen gaat het om het wonen in een accommodatie van een instelling met daarbij behorend toezicht en begeleiding, gericht op het bevorderen van zelfredzaamheid en participatie, het psychisch en psychosociaal functioneren, stabilisatie van een psychiatrisch ziektebeeld, het voorkomen van verwaarlozing of maatschappelijke overlast of het aanwenden van gevaar voor de cliënt of anderen, bestemd voor personen met psychische of psychosociale problemen, die niet in staat zijn zich op de eigen kracht te handhaven in de samenleving.</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-7\"><!----></a><span class=\"italic _p_nadrukcur\">Centrumgemeente:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-8\"><!----></a>Gemeente Doetinchem.</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-9\"><!----></a><span class=\"italic _p_nadrukcur\">Consulent Beschermd Wonen:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-10\"><!----></a>De medewerker in dienst bij de centrumgemeente, die voor de acht Achterhoekse gemeenten bepaalt of de maatwerkvoorziening Beschermd Wonen wordt ingezet. </p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-11\"><!----></a><span class=\"italic _p_nadrukcur\">Overbruggingszorg:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-12\"><!----></a>Onder overbruggingszorg wordt zorg verstaan, die ter overbrugging wordt geleverd in afwachting van de beschikbaarheid van een passende plek in een accommodatie van een instelling. Dit kan gelden voor een Beschermd Wonen plek als wel voor een plek in een instelling voor langdurige zorg.</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-13\"><!----></a><span class=\"italic _p_nadrukcur\">Psychisch:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-14\"><!----></a>een aandoening van geestelijke aard die aanwijsbaar is in DSM IV of DSM V (‘Diagnostic and Statistical Manual of Mental Disorders: het diagnostisch en statistisch handboek van psychiatrische aandoeningen).</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-15\"><!----></a><span class=\"italic _p_nadrukcur\">Pychosociaal:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-16\"><!----></a>hierbij gaat het om sociale en/of emotionele problemen die de leefsituatie aantoonbaar ernstig ontwrichten. </p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-17\"><!----></a><span class=\"italic _p_nadrukcur\">Sectorvreemde instelling:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-18\"><!----></a>een instelling, die langdurige zorg levert, maar niet specifiek bedoeld is voor het verblijf van GGZ-clienten. Het verblijf van cliënten, met een GGZ-C indicatie binnen een sectorvreemde instelling wordt bekostigd vanuit de centrumgemeente.</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-19\"><!----></a><span class=\"italic _p_nadrukcur\">Vertegenwoordiger:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-20\"><!----></a>een persoon of rechtspersoon, niet zijnde de betrokken zorgaanbieder, die een cliënt vertegenwoordigt die niet in staat kan worden geacht tot een redelijke waardering van zijn belangen ter zake;</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-21\"><!----></a><span class=\"italic _p_nadrukcur\">Volledig pakket thuis:</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-22\"><!----></a> een leveringsvorm in natura, waarbij de cliënt buiten de instelling ondersteuning in de thuissituatie ontvangt. </p><p class=\"al _p_al\"><a name=\"id1-3-2-2-1-23\"><!----></a><span class=\"italic _p_nadrukcur\">Wijkconsulent:</span> de medewerker die als consulent werkzaam is in één van de regiogemeenten.</p></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-2\"><!----></a><a name=\"id1-3-2-2-2-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-2-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">2</span> Melding hulpvraag</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-2-2-2\"><!----></a>Bij alle Achterhoekse gemeenten is een loket, waarbij de cliënt of zijn vertegenwoordiger of iemand uit het sociaal netwerk, zijn curator, zijn mentor of zijn gemachtigde zich kan melden voor Beschermd Wonen.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-2-3-2\"><!----></a>De Achterhoekse gemeenten analyseren voor hun inwoners welke lokale en regionale voorzieningen voor welk doel en resultaat nodig zijn. Wanneer Beschermd Wonen als regionale voorziening aan de orde is of kan zijn, dragen zij de benodigde screenings- en intakegegevens met toestemming van de cliënt over aan de centrumgemeente voor beoordeling en toekenning Beschermd Wonen.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-3\"><!----></a><a name=\"id1-3-2-2-3-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-3-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">3</span> Cliëntondersteuning</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-3-2\"><!----></a>Onafhankelijke cliëntondersteuning ten opzichte van de vraag van de cliënt wordt geboden door de buurtcoach of wijkconsulent en de consulent Beschermd Wonen. Stichting MEE Oost biedt 2016 onafhankelijke cliëntondersteuning gericht op bezwaar en beroep.</p></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-4\"><!----></a><a name=\"id1-3-2-2-4-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-4-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">4</span> Persoonlijk plan</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-4-2\"><!----></a>Bij het doen van een melding wordt de cliënt op de hoogte gesteld van de mogelijkheid een persoonlijk plan in te dienen. </p></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-5\"><!----></a><a name=\"id1-3-2-2-5-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-5-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">5</span> Informatie en identificatie</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-5-2-2\"><!----></a>De cliënt of zijn vertegenwoordiger, iemand uit het sociaal netwerk, zijn curator, zijn mentor, zijn gemachtigde of zijn hulpverlener verschaft de consulent Beschermd Wonen de gegevens en bescheiden die voor het onderzoek nodig zijn en waarover hij redelijkerwijs de beschikking kan krijgen.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-5-3-2\"><!----></a>De consulent Beschermd Wonen stelt de identiteit van de cliënt vast.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-6\"><!----></a><a name=\"id1-3-2-2-6-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-6-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">6</span> Onderzoek</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-6-2-2\"><!----></a>De consulent Beschermd Wonen, die de melding in behandeling heeft, bespreekt samen met de cliënt of zijn vertegenwoordiger, iemand uit het sociaal netwerk, zijn curator, zijn mentor of zijn gemachtigde wat de hulpvraag inhoudt en waaruit de behoefte aan ondersteuning bestaat.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-6-3-2\"><!----></a>Aan het gesprek kunnen indien door de cliënt gewenst de onafhankelijke cliëntondersteuner, de mantelzorger, de betrokken hulpverlener en derden deelnemen.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-6-4-2\"><!----></a>Het gesprek wordt weergegeven in een gespreksverslag, dat de cliënt wordt toegestuurd.</p></div></li><li class=\"_p_lid\"><span>4.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-6-5-2\"><!----></a>Het gespreksverslag kan als basis dienen voor de aanvraag van een individuele voorziening.</p></div></li><li class=\"_p_lid\"><span>5.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-6-6-2\"><!----></a>Bij twijfel tussen ambulante begeleiding en beschermd wonen wordt de lokale wijkconsulent betrokken.</p></div></li><li class=\"_p_lid\"><span>6.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-6-7-2\"><!----></a>De consulent Beschermd Wonen informeert tijdens het gesprek de cliënt over zijn rechten en plichten.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-7\"><!----></a><a name=\"id1-3-2-2-7-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-7-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">7</span> Advisering</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-7-2-2\"><!----></a>De consulent Beschermd Wonen bepaalt of medisch advies noodzakelijk is voor een goede beoordeling.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-7-3-2\"><!----></a>In het geval het vermoeden bestaat dat een voorliggende voorziening aan de orde is, kan de consulent Beschermd Wonen medisch advies vragen.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-7-4-2\"><!----></a>Medische gegevens kunnen met toestemming van de cliënt alleen door de consulent Beschermd Wonen worden opgevraagd.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-8\"><!----></a><a name=\"id1-3-2-2-8-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-8-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">8</span> Toelatingscriteria Beschermd Wonen</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-2\"><!----></a>De centrumgemeente stelt vast of de cliënt tot de doelgroep behoort waarvoor de maatwerkvoorziening Beschermd Wonen bedoeld is. Dit vindt plaats op basis van de volgende criteria, aan te halen als <span class=\"italic _p_nadrukcur\">‘toelatingscriteria Beschermd Wonen’:</span></p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-1-2\"><!----></a>Beschermd Wonen wordt toegekend wanneer de problemen die de cliënt ondervindt in het zelfstandig handhaven in de samenleving niet op te lossen zijn:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-1-3-1-2\"><!----></a>met een andere, voorliggende, voorziening, zoals de <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0018450\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0018450\">Zorgverzekeringswet</a> (Zvw), <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035917\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035917\">Wet langdurige zorg</a> (Wlz), <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0034925\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0034925\">Jeugdwet</a>, Wet forensische zorg of</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-1-3-2-2\"><!----></a>met eigen kracht, gebruikelijke hulp, mantelzorg, hulp van andere personen uit het sociale netwerk, gebruikmaking van algemeen gebruikelijke voorzieningen, algemene voorzieningen of voorliggende voorzieningen, para- of medische voorzieningen of extramurale begeleiding.</p></div></li></ul></div></li><li class=\"li _p_li\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-2\"><!----></a>De toepassing van Beschermd Wonen is aan de orde, indien aan alle volgende criteria wordt voldaan. Hierbij geldt de goedkoopst adequate oplossing als norm voor de te verstrekken voorziening.</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-3-1-2\"><!----></a>De cliënt 18 jaar is of ouder, kan zich niet zelfstandig handhaven in de samenleving en dit is niet op te lossen met de onder de 8.1 genoemde maatregelen;</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-3-2-2\"><!----></a>Een psychische aandoening of psychosociale beperking is door een ter zake kundige zoals een arts, psychiater, gz-psycholoog of verpleegkundig specialist vastgesteld;</p></div></li><li class=\"li _p_li\"><span>c.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-3-3-2\"><!----></a>Gebaseerd op de mogelijkheden van de cliënt, staat de op participatie gerichte ondersteuning vanuit de beschermde woonomgeving op de voorgrond;</p></div></li><li class=\"li _p_li\"><span>d.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-3-4-2\"><!----></a>Er is geen sprake van een acute situatie in de geestelijke gezondheid of op andere levensdomeinen;</p></div></li><li class=\"li _p_li\"><span>e.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-3-5-2\"><!----></a>Het verblijf in een instelling met daarbij behorende samenhangende ondersteuning is noodzakelijk: voortdurend in de nabijheid of 24 uur per dag toezicht en intensieve ongeplande of geplande dagelijkse begeleiding;</p></div></li><li class=\"li _p_li\"><span>f.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-2-3-6-2\"><!----></a>De cliënt accepteert - op termijn - een begeleidings- of ontwikkelingstraject dat, met inachtneming van zijn mogelijkheden, gericht is op het realiseren van een situatie, waarin hij in staat wordt gesteld zich zo snel mogelijk weer op eigen kracht te handhaven in de samenleving.</p></div></li></ul></div></li><li class=\"li _p_li\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-8-3-3-2\"><!----></a>De cliënt, die in aanmerking komt voor Beschermd Wonen, komt niet in aanmerking voor een aparte maatwerkvoorziening gericht op ondersteuning bij het huishouden.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-9\"><!----></a><a name=\"id1-3-2-2-9-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-9-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">9</span> Toegang in relatie tot regiobinding</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-2-2\"><!----></a>De cliënt, die is aangewezen op Beschermd Wonen, krijgt deze ondersteuning in de gemeente waar hij de meeste kans op zelfredzaamheid, participatie, herstel en ontwikkeling heeft. Dit wordt regiobinding genoemd. Dit wordt door de centrumgemeente vastgesteld aan de hand van in 9.2. genoemde criteria.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-3-2\"><!----></a>Toekenning tot een Beschermd Wonen-plek binnen de regio Achterhoek is aan de orde wanneer gegronde redenen zijn om tegemoet te komen aan de wens van de cliënt om in een bepaalde Achterhoekse gemeente beschermd te willen wonen. </p><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-3-3\"><!----></a>Hiervoor moet aan één of meer van de volgende criteria worden voldaan:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-3-4-1-2\"><!----></a>In de regio Achterhoek bestaat een passend zorgaanbod dat in de regio van herkomst niet leverbaar is;</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-3-4-2-2\"><!----></a>Er is een aantoonbaar positief sociaal netwerk, te weten: minimaal één familielid of vriend binnen de regio Achterhoek;</p></div></li><li class=\"li _p_li\"><span>c.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-3-4-3-2\"><!----></a>Gegronde redenen om cliënt uit zijn oude netwerk te halen mede gesteld door zijn huisarts, verwijzer of huidige aanbieder;</p></div></li><li class=\"li _p_li\"><span>d.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-9-3-4-4-2\"><!----></a>Aanmerkelijk belang in verband met - perspectief op - een opleiding, vrijwilligerswerk of betaald werk binnen de regio Achterhoek.</p></div></li></ul></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-10\"><!----></a><a name=\"id1-3-2-2-10-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-10-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">10</span> Woonverblijf in een sectorvreemde instelling</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-10-2\"><!----></a>De centrumgemeente kiest ervoor om cliënten, die beschermd wonen binnen een sectorvreemde instelling, deze plek te laten behouden, mits deze plek de meest passende vorm is.</p></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-11\"><!----></a><a name=\"id1-3-2-2-11-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-11-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">11</span> Toekenning, resultaat, omvang en duur</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-11-2-2\"><!----></a>De centrumgemeente is verantwoordelijk voor het aanbieden van Beschermd Wonen. Wanneer een cliënt in aanmerking komt voor Beschermd Wonen, zorgt de centrumgemeente voor de beschikking en levering van Beschermd Wonen.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-11-3-2\"><!----></a>In de beschikking wordt het resultaat omschreven waartoe het bieden van Beschermd Wonen moet leiden.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-11-4-2\"><!----></a>In de beschikking wordt aangegeven of de maatwerkvoorziening in natura of als persoonsgebonden budget wordt verstrekt.</p></div></li><li class=\"_p_lid\"><span>4.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-11-5-2\"><!----></a>De omvang van Beschermd Wonen is voor het jaar 2016 gebaseerd op zes zorgzwaartepakketten:</p><div class=\"_p_table\"><a name=\"id1-3-2-2-11-5-3\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-1-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">pakket</span></p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-1-2-1\"><!----></a><span class=\"vet _p_nadrukvet\">Functie-omschrijving</span></p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-2-1-1\"><!----></a>1 GGZ-C</p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-2-2-1\"><!----></a>Beschermd wonen met begeleiding</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-3-1-1\"><!----></a>2 GGZ-C</p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-3-2-1\"><!----></a>Gestructureerd beschermd wonen met uitgebreide begeleiding</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-4-1-1\"><!----></a>3 GGZ-C</p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-4-2-1\"><!----></a>Beschermd wonen met intensieve begeleiding</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-5-1-1\"><!----></a>4 GGZ-C</p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-5-2-1\"><!----></a>Gestructureerd beschermd wonen met intensieve begeleiding en verzorging</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-6-1-1\"><!----></a>5 GGZ-C</p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-6-2-1\"><!----></a>Beschermd wonen met intensieve begeleiding en gedragsregulering</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-7-1-1\"><!----></a>6 GGZ-C</p></td><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-2-11-5-3-1-3-7-2-1\"><!----></a>Beschermd wonen met intensieve begeleiding en intensieve verpleging en verzorging</p></td></tr></tbody></table></div><p class=\"al _p_al\"><a name=\"id1-3-2-2-11-5-4\"><!----></a>Deze pakketten kunnen zowel met als zonder dagbesteding of vervoer worden geïndiceerd</p></div></li><li class=\"_p_lid\"><span>5.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-11-6-2\"><!----></a>De toekenning voor Beschermd Wonen is voor maximaal twee jaar. Dit geldt voor zowel een verstrekking in natura als voor een verstrekking in de vorm van een persoonsgebonden budget.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-12\"><!----></a><a name=\"id1-3-2-2-12-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-12-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">12</span> Tijdelijk verblijf elders</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-12-2-2\"><!----></a>Indien een cliënt, tijdelijk als gevolg van een behandeling in een ziekenhuis of behandelcentrum of detentie elders wordt opgenomen, wordt de beschermde woonplek voor een maximum van 14 dagen door de centrumgemeente bekostigd als ware de cliënt verblijft in de Beschermd Wonen instelling. Vervolgens mag een zorgaanbieder die zorg in natura aanbiedt, gedurende maximaal vier weken de verblijfscomponent nog declareren. In het Besluit maatschappelijke ondersteuning gemeente Doetinchem is hiervoor het bedrag per dag opgenomen. Bij levering via een persoonsgebonden budget is nimmer sprake dat de verblijfscomponent uit het persoonsgebonden budget mag worden bekostigd.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-12-3-2\"><!----></a>De gecontracteerde aanbieder van Beschermd Wonen, voor zorg in natura, is verplicht om elke tijdelijke opname elders binnen een week te melden bij de centrumgemeente.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-12-4-2\"><!----></a>In het geval de termijn zoals in artikel 12.2 dreigt te worden overschreden, dient minimaal een week voor het verlopen van de termijn contact opgenomen te worden met de centrumgemeente. Vervolgens zal door de consulent Beschermd Wonen worden bepaald of een maatwerkoplossing noodzakelijk is.</p></div></li><li class=\"_p_lid\"><span>4.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-12-5-2\"><!----></a>De cliënt, die via een persoonsgebonden budget Beschermd Wonen ontvangt of zijn vertegenwoordiger of iemand uit het sociaal netwerk, zijn curator, zijn mentor of zijn gemachtigde, is verplicht om elke tijdelijke opname elders terstond te melden aan de consulent Beschermd Wonen.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-13\"><!----></a><a name=\"id1-3-2-2-13-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-13-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">13</span> Overbruggingszorg</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-13-2-2\"><!----></a>Wanneer intramuraal verblijf niet direct beschikbaar is, dan is de centrumgemeente verantwoordelijk voor het bieden van een passend alternatief voor overbruggingszorg in afwachting van de beschikbaarheid van een passende plek Beschermd Wonen in een accommodatie van een instelling.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-13-3-2\"><!----></a>Wanneer een indicatie voor de <a href=\"http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035917\" title=\"link naar publicatie http://wetten.overheid.nl/cgi-bin/deeplink/law1/bwbid=BWBR0035917\">Wet langdurige zorg</a> is toegekend, komt de bekostiging van de overbruggingszorg ten laste van de Wet langdurige zorg.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-13-4-2\"><!----></a>Overbruggingszorg kan ook via een persoonsgebonden budget worden ingezet. Wanneer overbruggingszorg via een persoonsgebonden budget wordt ingezet gelden de volgende uitgangspunten:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-13-4-3-1-2\"><!----></a>een persoonsgebonden budget als overbruggingszorg voor thuiswonenden wordt aangepast op de situatie thuis;</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-13-4-3-2-2\"><!----></a>de hoogte van een persoonsgebonden budget ter overbrugging wordt bepaald op basis van de daadwerkelijk te maken kosten om de begeleiding en bescherming goed te organiseren.</p></div></li></ul></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-14\"><!----></a><a name=\"id1-3-2-2-14-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-14-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">14</span> Wat niet wordt vergoed vanuit een persoonsgebonden budget</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-14-2-2\"><!----></a>Er wordt geen toeslag op een persoonsgebonden budget toegekend voor kleinschalige wooninitiatieven;</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-14-3-2\"><!----></a>De gemeente verstrekt geen eenmalige uitkering in het geval de zorgovereenkomst van rechtswege wordt beëindigd.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-14-4-2\"><!----></a>Kosten die de aanbieder in rekening brengt in verband met een opzegtermijn zijn niet te verhalen op de gemeente. Ook kosten die de aanbieder de cliënt in rekening brengt voor het niet nakomen van een afspraak kunnen niet worden verhaald op de gemeente.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-15\"><!----></a><a name=\"id1-3-2-2-15-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-15-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">15</span> Wijze van verstrekking</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-15-2\"><!----></a>Beschermd Wonen wordt toegekend in:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-15-3-1-2\"><!----></a>de vorm van een voorziening in natura of;</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-15-3-2-2\"><!----></a>de vorm van een persoonsgebonden budget.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-16\"><!----></a><a name=\"id1-3-2-2-16-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-16-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">16</span> Niet verzilvering</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-16-2-2\"><!----></a>Een persoonsgebonden budget dient volgens <a href=\"http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/362927/362927_1.html\" title=\"link naar publicatie http://decentrale.regelgeving.overheid.nl/cvdr/XHTMLoutput/Historie/Montferland/362927/362927_1.html\">artikel 6.1 lid 2 onder c. van de Verordening</a> door de cliënt binnen zes maanden na toekenning te worden aangewend ten behoeve van het resultaat waarvoor het is verstrekt. Deze periode van zes maanden hanteert de centrumgemeente ook voor een voorziening in natura.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-16-3-2\"><!----></a>Indien de genoemde termijn in artikel 16 onder 1 dreigt te worden overschreden wordt de consulent Beschermd Wonen door de cliënt of zijn vertegenwoordiger of de zorgaanbieder minimaal vier weken voorafgaand aan het verlopen van de geldigheidsduur van de maatwerkvoorziening op de hoogte gesteld.</p></div></li><li class=\"_p_lid\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-16-4-2\"><!----></a>Indien de maatwerkvoorziening niet binnen zes maanden wordt aangewend, dan vervalt de toekenning.</p></div></li><li class=\"_p_lid\"><span>4.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-16-5-2\"><!----></a>Indien sprake is van een gedeeltelijk gebruik van het persoonsgebonden budget wordt de consulent Beschermd Wonen door de cliënt of zijn vertegenwoordiger of zijn hulpverlener minimaal vier weken voorafgaand aan het verlopen van de geldigheidsduur van de maatwerkvoorziening op de hoogte gesteld.</p></div></li></ul></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-17\"><!----></a><a name=\"id1-3-2-2-17-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-17-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">17</span> Voorkoming oneigenlijk gebruik en misbruik</p><p class=\"al _p_al\"><a name=\"id1-3-2-2-17-2\"><!----></a>De centrumgemeente neemt alle maatregelen, die nodig zijn ter voorkoming van oneigenlijk gebruik en misbruik van de toegekende voorziening. </p></div><div class=\"_p_artikel\"><a name=\"id1-3-2-2-18\"><!----></a><a name=\"id1-3-2-2-18-1\"><!----></a><p class=\"artikel_kop _p_artikel_kop_titel\"><a name=\"id1-3-2-2-18-1-3\"><!----></a><span class=\"artikel_kop _p_artikel_kop_label\">Artikel</span> <span class=\"artikel_kop _p_artikel_kop_nr\">18</span> Inwerkingtreding en citeertitel</p><ul class=\"_explicit\"><li class=\"_p_lid\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-18-2-2\"><!----></a>Deze beleidsregels treden in werking op 1 januari 2016.</p></div></li><li class=\"_p_lid\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-2-18-3-2\"><!----></a>Deze beleidsregels worden aangehaald als: </p><p class=\"al _p_al\"><a name=\"id1-3-2-2-18-3-3\"><!----></a>Beleidsregels Beschermd Wonen Montferland 2016.</p></div></li></ul></div></div><div class=\"_p_regeling-sluiting\"><a name=\"id1-3-2-3\"><!----></a><div class=\"_p_slotformulering\"><a name=\"id1-3-2-3-1\"><!----></a><p class=\"al _p_al\"><a name=\"id1-3-2-3-1-1\"><!----></a> </p></div><div class=\"_p_gegeven\"><a name=\"id1-3-2-3-2\"><!----></a><p class=\"dagtekening _p_dagtekening\"><a name=\"id1-3-2-3-2-1\"><!----></a><span class=\"_p_datum\">Aldus vastgesteld te Didam op 10 mei 2016</span></p></div><div style=\"font-style: italic; line-height: 130%;\"><p>Burgemeester en wethouders van de gemeente Montferland,</p><!----></div><div style=\"font-style: italic; line-height: 130%;\"><p>de secretaris,</p><!----></div><div style=\"font-style: italic; line-height: 130%;\"><p>T.M.J.M. Evers</p><!----></div><div style=\"font-style: italic; line-height: 130%;\"><p>de burgemeester,</p><!----></div><div style=\"font-style: italic; line-height: 130%;\"><p>mw. C.C. Leppink-Schuitema</p><!----></div></div><div class=\"_p_nota-toelichting\"><a name=\"id1-3-2-4\"><!----></a><a name=\"id1-3-2-4-1\"><!----></a><div class=\"artikel_kop\"><span class=\"_p_label\">Toelichting</span> </div><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-2\"><!----></a>Algemene toelichting</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-3\"><!----></a>De wetgever heeft de uitvoering van de Wet maatschappelijke ondersteuning en specifiek voor het onderdeel Beschermd Wonen opgedragen aan de centrumgemeenten.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-4\"><!----></a>De centrumgemeente Doetinchem (verder te noemen: centrumgemeente) is (materieel) verantwoordelijk voor Beschermd Wonen binnen de regio Achterhoek. Hiertoe behoren de gemeenten: Aalten, Berkelland, Bronckhorst, Doetinchem, Montferland, Oost Gelre, Oude IJsselstreek en Winterswijk (hierna: de Achterhoekse gemeenten).</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-5\"><!----></a>De beleidsregels Beschermd Wonen is het met deze gemeenten afgesproken kader waarbinnen de centrumgemeente de bevoegdheid tot het beoordelen en toekennen van Beschermd Wonen binnen de regio Achterhoek dient uit te voeren.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-6\"><!----></a>De beleidsregels Beschermd Wonen regio Achterhoek 2016 zijn de nadere uitwerking van de Verordening op het gebied van melding hulpvraag, cliëntondersteuning, het persoonlijk plan, informatie en identificatie, het onderzoek, de advisering, de toelatingscriteria, toegang in relatie tot regiobinding, woonverblijf in een sectorvreemde instelling, toekenning, resultaat, omvang en duur, tijdelijk verblijf elders, overbruggingszorg, wijze van verstrekking, niet verzilvering en voorkoming van oneigenlijk gebruik en misbruik. De nummering van de artikelen in deze beleidsregels komt niet overeen met de nummering van de artikelen in de Verordening. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-7\"><!----></a>De Wet maatschappelijke ondersteuning 2015 gaat uit van maatwerk. Om toch enige eenheid te brengen in de uitvoering zijn deze beleidsregels opgesteld. De beleidsregels geven aan op welke wijze het college van haar bevoegdheid gebruik maakt. Deze regels “binden” daarmee het college. Het wordt daarmee voor elke inwoner transparant op welke manier door het college invulling wordt gegeven aan de hulpvraag. De beleidsregels zijn niet volledig dekkend teneinde maatwerk mogelijk te maken. </p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-8\"><!----></a>Artikelsgewijze toelichting</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-9\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 1 Begripsbepalingen</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-10\"><!----></a>De definitie ‘<span class=\"italic _p_nadrukcur\">Beschermd Wonen’</span> is rechtstreeks ontleend aan de Wet maatschappelijke ondersteuning 2015.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-11\"><!----></a>Beschermd Wonen is een passende maatwerkvoorziening voor personen van achttien jaar of ouder, die als gevolg van een psychische of psychosociale aandoening niet in staat zijn zich op eigen kracht te handhaven in de samenleving. Beschermd Wonen is voor mensen die als gevolg van deze aandoening 24 uurs toezicht of begeleiding nodig hebben. De ondersteuning is gericht op het leren functioneren met deze al dan niet blijvende beperking, met als doelstelling zoveel mogelijk zelfredzaamheid en participatie in de maatschappij te bereiken.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-12\"><!----></a>Bij Beschermd Wonen staat hulp op maat centraal. Wanneer Beschermd Wonen een te zwaar zorgpakket is voor de behoeftes van het individu, dan is er de maatwerkvoorziening ambulante begeleiding zoals overnemen, coachen of basis, middel of zwaar.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-13\"><!----></a>Kent de Wet maatschappelijke ondersteuning 2015 geen grondslagen voor de ondersteuningsbehoefte, voor Beschermd Wonen maken wij hierop een uitzondering. Bij Beschermd Wonen is een vastgestelde psychische of psychosociale aandoening een hard criterium. Mensen die als gevolg van een psychische aandoening zodanig beperkt zijn in hun zelfredzaamheid dat ze niet zonder 24-uurs toezicht of begeleiding kunnen, vormen een bijzonder kwetsbare groep. Hoewel de ondersteuning uiteraard zoveel mogelijk is gericht op het verbeteren van zelfredzaamheid en participatie, is een beschermde omgeving waarin zij zich veilig voelen en hen gespecialiseerde ondersteuning geboden wordt onontbeerlijk voor hun functioneren en herstel. Beschermd Wonen is dan ook niet bedoeld voor personen met psychosociale problemen die met eigen inzet oplosbaar zijn of voor mensen die in de eerste plaats een woonprobleem hebben. Daarbij is de beschermde component niet in lijn met de ondersteuningsbehoefte. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-14\"><!----></a>Deze beleidsregels maken gebruik van de termen psychisch en psychosociaal. Aangezien deze termen niet in bovenliggende wetgeving (Wmo 2015) zijn gedefinieerd, vullen deze beleidsregels de termen in. Waar in deze beleidsregels “psychisch” als term wordt gebruikt, gaat het om een aandoening van geestelijke aard die aanwijsbaar is in DSM-IV of DSM-V (‘Diagnostic and Statistical Manual of Mental Disorders: het diagnostisch en statistisch handboek van psychiatrische aandoeningen).</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-15\"><!----></a>De term psychosociaal is complexer van aard. Het gaat daarbij om sociale en/of emotionele problemen die de leefsituatie aantoonbaar ernstig ontwrichten, bijvoorbeeld een combinatie van schulden, middelengebruik en overlastgevend gedrag of ernstige relatie- en opvoedingsproblemen in combinatie met economische onzelfstandigheid. Psychosociale problemen hebben een psychische component, bijvoorbeeld ernstige stress, angst of agressieregulatieproblemen. Deze problemen, alhoewel niet definieerbaar als een psychische aandoening op grond van DSM-IV of DSM-V, kunnen wel reden zijn dat een maatwerkvoorziening nodig is waarbij de regie tijdelijk geheel of grotendeels moet worden overgenomen. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-16\"><!----></a>Een vertegenwoordiger is op grond van de definitie van de Wmo 2015 de persoon of rechtspersoon die een cliënt die niet in staat kan worden geacht tot een redelijke waardering van zijn belangen ter zake vertegenwoordigt.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-17\"><!----></a>Op grond van de wet kunnen als vertegenwoordiger optreden de curator, mentor of gevolmachtigde van de cliënt. De bewindvoerder staat hier niet bij. De bewindvoerder kan alleen als vertegenwoordiger optreden als deze is gevolmachtigd door de cliënt.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-18\"><!----></a>Curatele, bewind en mentorschap zijn verschillende maatregelen om mensen, die zelf geen goede beslissingen kunnen nemen te beschermen. Bijvoorbeeld vanwege een verstandelijke beperking, verslaving of dementie. Bewind is bedoeld voor wie zijn financiële zaken zelf niet kan regelen. Mentorschap gaat over het nemen van beslissingen over de verzorging, verpleging, behandeling en begeleiding van de cliënt. Curatele is bedoeld voor mensen die hun financiële én persoonlijke zaken zelf niet kunnen regelen. Die mensen zijn handelingsonbekwaam. Curatele, bewind en mentorschap kunnen worden aangevraagd bij de kantonrechter.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-19\"><!----></a>Als een curator, mentor of gevolmachtigde ontbreekt, kan ook als vertegenwoordiger optreden:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-20-1-2\"><!----></a>Echtgenoot, geregistreerd partner of andere levensgezel van de cliënt; dan wel (als deze ontbreekt);</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-20-2-2\"><!----></a>Zijn ouder, (meerderjarig) kind, broer of zus.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-21\"><!----></a>Deze personen kunnen echter niet als een vertegenwoordiger optreden als de cliënt dat niet wenst. Om die reden is het gewenst dat een schriftelijke machtiging van de cliënt wordt overlegd. Een buurvrouw, vriend of kennis, kan alleen als vertegenwoordiger optreden als deze expliciet door de cliënt is gevolmachtigd.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-22\"><!----></a>Met een ‘<span class=\"italic _p_nadrukcur\">Volledig pakket thuis’</span> kan een cliënt thuis de benodigde ondersteuning ontvangen, die hij ook in een instelling zou krijgen. Er dient sprake te zijn van 24 uurs toezicht in de nabijheid. De wooncomponent valt buiten het volledig pakket thuis: kosten van huur of hypotheek is voor rekening cliënt. De mogelijkheid voor het verstrekken van een ‘<span class=\"italic _p_nadrukcur\">Volledig pakket thuis’</span> geldt voor het jaar 2016. Zoals in de begripsbepaling is aangegeven geldt het ‘Volledig pakket thuis’ alleen voor levering in natura. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-23\"><!----></a>Bij een ‘Volledig pakket thuis’ ontvangt de cliënt een compleet zorgpakket thuis van één aanbieder. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-24\"><!----></a>Het ‘Volledig pakket thuis’ bevat bijna alle zorg die een cliënt ook in een instelling zou krijgen, maar dan thuis. Dat houdt in:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-25-1-2\"><!----></a>verpleging</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-25-2-2\"><!----></a>persoonlijke verzorging</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-25-3-2\"><!----></a>begeleiding (individueel en/of dagbesteding) </p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-25-4-2\"><!----></a>vervoer naar behandeling en/of dagbesteding;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-25-5-2\"><!----></a>maaltijden (eten en drinken);</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-25-6-2\"><!----></a>hulp bij het huishouden. </p></div></li></ul><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-26\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 2 Melding </span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-27\"><!----></a>De vraag om Beschermd Wonen kan binnen elke Achterhoekse gemeente binnenkomen. De wijze waarop is afhankelijk van de lokale inrichting: via het sociale team, het wijkteam, het Wmo-loket, het Zorgplein, de buurtcoach etc. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-28\"><!----></a>De Achterhoekse gemeenten analyseren voor hun inwoners welke lokale en regionale voorzieningen voor welk doel en resultaat nodig zijn. Als blijkt dat Beschermd Wonen aan de orde is of kan zijn, worden met toestemming van de cliënt de gegevens zo spoedig mogelijk overgedragen naar de centrumgemeente. Gegevensuitwisseling vindt plaats met inachtneming van de bepalingen van de Wet bescherming persoonsgegevens. In bepaalde gevallen is het mogelijk dat de consulent van de regiogemeente samen met de consulent Beschermd Wonen van de centrumgemeente het onderzoek verricht.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-29\"><!----></a>Essentiële informatie die bij de melding verstrekt dient te worden:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-30-1-2\"><!----></a>Indien aanwezig documenten waarin een beschrijvende diagnose of een DSM-IV of DSM-V classificatie is opgenomen en algemene informatie inzake de diagnose (naam terzake deskundige en datum waarop diagnose gesteld is);</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-30-2-2\"><!----></a>Document waarin ziekteverloop, de behandelgeschiedenis en bijkomende problematiek wordt weergegeven.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-31\"><!----></a>Een cliënt kan een aanvraag Beschermd Wonen indienen vanaf acht weken voordat hij achttien jaar wordt. </p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-32\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 3 Cliëntondersteuning</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-33\"><!----></a>Op basis van de Wmo 2015 is de gemeente verantwoordelijk voor cliëntondersteuning aan alle inwoners. Cliëntondersteuning wordt in de Wmo 2015 geformuleerd als het bieden van informatie, advies en algemene ondersteuning, die bijdraagt aan het versterken van de zelfredzaamheid en participatie. Wanneer een cliënt zich meldt met een hulpvraag, dus bij de 'toegang' tot ondersteuning, moet hij of zij gebruik kunnen maken van cliëntondersteuning. Hiervoor kan iemand uit het sociale netwerk, zoals familie of vrienden ingezet worden, maar het kan ook een professionele hulpverlener zijn. Cliëntondersteuning begint bij een goede toegang, zoals verzorgd door de consulent Beschermd Wonen. De onafhankelijkheid van de cliëntondersteuning wordt gewaarborgd door de professionele autonomie en attitude van de consulent Beschermd Wonen. Hij kijkt dus onafhankelijk naar de vraag. Het kan zijn dat de consulent Beschermd Wonen en de cliënt het niet eens worden. In dat geval heeft de centrumgemeente voor de cliënt de volgende mogelijkheden voor cliëntondersteuning: </p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-34-1-2\"><!----></a>Als de cliënt twijfelt aan de objectiviteit, onafhankelijkheid of attitude van de buurtcoach, wijkconsulent of de consulent Beschermd Wonen, kan de cliënt een andere buurtcoach, wijkconsulent of consulent Beschermd Wonen vragen. De betreffende teamleider van het Buurtplein, Zorgplein of regiogemeente bepaalt in samenspraak met de cliënt of een andere medewerker wordt ingezet.</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-34-2-2\"><!----></a>Stichting MEE Oost is door de Achterhoekse gemeenten, met uitzondering van de gemeente Oude IJsselstreek voor 2016 gecontracteerd voor onafhankelijke cliëntondersteuning. De afspraken die iedere gemeente met Stichting MEE Oost hebben gemaakt, kunnen van elkaar verschillen.  </p></div></li></ul><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-35\"><!----></a>Artikel 4 Persoonlijk plan</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-36\"><!----></a>De cliënt wordt door degene die de melding in ontvangst neemt geïnformeerd over de mogelijkheid om binnen zeven dagen een persoonlijk plan te overhandigen. Daarin beschrijft hij de onderwerpen die tijdens het onderzoek aan de orde zullen komen. De cliënt geeft in het plan aan welke problemen hij ondervindt en welke vorm van maatschappelijke ondersteuning naar zijn mening het meest passend is. </p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-37\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 5 Informatie en identificatie</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-38\"><!----></a>De cliënt dient een geldig identiteitsbewijs te overleggen en toestemming te verlenen voor de verwerking van persoonsgegevens waarover de gemeente uit andere hoofde beschikt, bijvoorbeeld ten behoeve van uitvoering van de Jeugdwet, de Participatiewet en de Wet gemeentelijke schuldhulpverlening, of die de gemeente van de zorgverzekeraar, de zorgaanbieder of andere derden zou willen ontvangen en die noodzakelijk zijn voor de uitvoering van de Wmo 2015.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-39\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 6 Onderzoek</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-40\"><!----></a>De consulent Beschermd Wonen onderzoekt in het gesprek met de cliënt wat zijn ondersteuningsbehoefte is. Het streven is dat dit gesprek binnen twee weken na het eerste contact plaatsvindt. Spoedgevallen worden uiteraard met voorrang behandeld. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-41\"><!----></a><span class=\"italic _p_nadrukcur\">Het gesprek</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-42\"><!----></a>Het gesprek vindt plaats met cliënt en zijn vertegenwoordiger of iemand uit het sociaal netwerk, zijn mentor, zijn gemachtigde of zijn hulpverlener. Het gesprek vormt de basis voor het onderzoek en wordt bij voorkeur door de consulent Beschermd Wonen bij de cliënt thuis gevoerd. Het gesprek heeft een tweeledig doel, namelijk: </p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-43-1-2\"><!----></a>De vraagverheldering en het vaststellen van het te behalen resultaat; </p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-43-2-2\"><!----></a>Het afspreken van oplossingen; het ondersteuningsarrangement vaststellen. </p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-44\"><!----></a>Van belang is om eerst in kaart te brengen wat de ondersteuningsbehoefte en de specifieke situatie van de cliënt zijn, voordat er over oplossingen gesproken wordt. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-45\"><!----></a>De cliënt kan zich bij dit gesprek laten ondersteunen door anderen, bijvoorbeeld een mantelzorger, hulpverlener of een onafhankelijke cliëntondersteuner. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-46\"><!----></a>Tijdens het gesprek komt de specifieke situatie van de cliënt uitgebreid aan bod. Relevante persoonskenmerken kunnen, afhankelijk van de belemmeringen die de cliënt aandraagt, bijvoorbeeld zijn:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-47-1-2\"><!----></a>de leeftijd;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-47-2-2\"><!----></a>de gezondheidssituatie;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-47-3-2\"><!----></a>de mate waarin de cliënt actief is geweest voordat hij geconfronteerd werd met zijn beperkingen;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-47-4-2\"><!----></a>de zelfstandigheid en leerbaarheid van de cliënt.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-48\"><!----></a>De behoefte aan maatregelen ter ondersteuning van de mantelzorger van de cliënt wordt nadrukkelijk meegenomen in het gesprek. Ook wordt de cliënt gewezen op zijn rechten en plichten.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-49\"><!----></a>De onderzoeksfase is een waarborg voor de cliënt om gehoord te worden en in gezamenlijk overleg tot een kwalitatief goed ondersteuningsplan te komen. Hierbij wordt ook aandacht besteed aan de eigen mogelijkheden van de cliënt. Bovendien is het van belang na te gaan of de voorziening/ondersteuning die de cliënt voor ogen heeft, geen schijnoplossing is voor mogelijk onderliggende problematiek, de vraag achter de vraag.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-50\"><!----></a><span class=\"italic _p_nadrukcur\">Het gespreksverslag</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-51\"><!----></a>Van het gesprek wordt door de consulent Beschermd Wonen een gespreksverslag gemaakt. Dit gespreksverslag bestaat uit een schriftelijke weergave van het gesprek en de afspraken die eruit zijn voortgekomen. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-52\"><!----></a>De cliënt of vertegenwoordiger ontvangt hiervan een afschrift. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-53\"><!----></a>In het gesprek kan blijken dat er eigenlijk geen noodzaak bestaat voor maatwerkondersteuning, maar men al geholpen is met een oplossing binnen het eigen netwerk of met een verwijzing naar een algemene voorziening. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-54\"><!----></a>De uitkomsten van het onderzoek kunnen echter ook leiden tot een ondersteuningsvraag. Voor de ondersteuningsvraag wordt samen met de consulent Beschermd Wonen naar een passend ondersteuningsarrangement of een andere wettelijke voorziening gezocht. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-55\"><!----></a><span class=\"italic _p_nadrukcur\">Informeren over rechten en plichten</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-56\"><!----></a>De consulent Beschermd Wonen informeert de cliënt over de mogelijkheid zich te laten ondersteunen door een cliëntondersteuner en ondersteuning van anderen; de wens van de cliënt staat hierbij centraal.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-57\"><!----></a>Wanneer een maatwerkvoorziening aangewezen is, informeert de consulent Beschermd Wonen de client of zijn vertegenwoordiger over het door de gemeente gecontracteerde aanbod. Ook informeert de consulent Beschermd Wonen de cliënt over de mogelijkheid om te kiezen voor een verstrekking van een persoonsgebonden budget. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-58\"><!----></a>De consulent Beschermd Wonen wijst de cliënt of vertegenwoordiger in het gesprek expliciet op de mogelijkheid om zijn bekwaamheid met betrekking tot het omgaan met een persoonsgebonden budget, te testen. Het testen is bedoeld om de cliënt of vertegenwoordiger bewust te maken van de taken en verantwoordelijkheden die horen bij het beheren van een persoonsgebonden budget. Hiermee kan hij een weloverwogen keuze voor persoonsgebonden budget of zorg in natura maken.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-59\"><!----></a>In het geval er een maatwerkvoorziening wordt ingezet informeert de consulent Beschermd Wonen de cliënt of vertegenwoordiger over de verschuldigde eigen bijdrage en hoe de hoogte daarvan vastgesteld en geïnd wordt. </p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-60\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 7 Advisering</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-61\"><!----></a>Advies kan worden gevraagd wanneer het een eerste melding betreft, bij twijfel over de medische noodzaak of bij een voorgenomen afwijzing van een maatwerkvoorziening. Ook kan een medisch advies zinvol zijn om te beoordelen wat de dominante grondslag is en voor het in kaart brengen van de behandel- en ontwikkelingsmogelijkheden van de cliënt. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-62\"><!----></a>De Algemene wet bestuursrecht (Awb) geeft enige algemene bepalingen over (externe) advisering. Onder adviseur wordt verstaan: een persoon of college, bij of krachtens wettelijk voorschrift belast met het adviseren inzake door een bestuursorgaan te nemen besluiten en niet werkzaam onder verantwoordelijkheid van dat bestuursorgaan. De advisering zal verder altijd plaatsvinden door een onafhankelijke organisatie die niet aan de gemeente verbonden is. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-63\"><!----></a>Er worden niet meer gegevens opgevraagd dan noodzakelijk voor het nemen van een besluit op de aanvraag. Ten aanzien van het omgaan met de – vaak privacygevoelige – gegevens houden we rekening met de verplichtingen die voortvloeien uit de Wet bescherming persoonsgegevens (Wbp). De kosten van het medisch advies zijn voor rekening van de centrumgemeente. In het geval dat wordt afgeweken van een medisch advies, moet in de beschikking worden gemotiveerd waarom hiervan wordt afgeweken.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-64\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 8 Toelatingscriteria Beschermd Wonen</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-65\"><!----></a>Als centrumgemeente kiezen we ervoor om aan de hand van elf domeinen van het dagelijks leven de mate van zelfredzaamheid te beoordelen. Deze levensdomeinen zijn respectievelijk:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-1-2\"><!----></a>financiën;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-2-2\"><!----></a>dagbesteding;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-3-2\"><!----></a>huisvesting;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-4-2\"><!----></a>huiselijke relaties;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-5-2\"><!----></a>geestelijke gezondheid;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-6-2\"><!----></a>lichamelijke gezondheid;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-7-2\"><!----></a>verslaving;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-8-2\"><!----></a>Activiteiten Dagelijks Leven: afgekort ADL;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-9-2\"><!----></a>sociaal netwerk; </p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-10-2\"><!----></a>maatschappelijke participatie;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-66-11-2\"><!----></a>justitie.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-67\"><!----></a>Voor het bieden van Beschermd Wonen kan aanleiding bestaan indien iemand er vanwege psychische of psychosociale problematiek niet in slaagt om zelfstandig te wonen zonder de directe nabijheid van 24 uur per dag toezicht of ondersteuning. En indien deze niet met eigen kracht, met steun van de omgeving, met een algemene voorziening of maatwerkvoorziening voldoende geholpen is.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-68\"><!----></a><span class=\"italic _p_nadrukcur\">Eigen kracht</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-69\"><!----></a>De centrumgemeente wil <span class=\"italic _p_nadrukcur\">‘eigen kracht’</span> insteken vanuit het welbevinden en mogelijkheden van de cliënt; door niet het probleem met de cliënt te bespreken, maar juist te bespreken wat de cliënt kan. <span class=\"italic _p_nadrukcur\">‘Eigen kracht’</span> wordt daarmee op een positieve manier uitgangspunt van het gesprek. Eventuele compensatie vanuit de centrumgemeente komt daarmee pas in een later stadium in het gesprek aan bod. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-70\"><!----></a>De ‘<span class=\"italic _p_nadrukcur\">eigen kracht’</span> van de cliënt heeft betrekking op de mogelijkheden van de cliënt om zelf bij te dragen aan zijn zelfredzaamheid en participatie. De <span class=\"italic _p_nadrukcur\">eigen kracht’</span> komt niet alleen tot uitdrukking op het moment dat de cliënt al belemmeringen heeft, maar ook daarvoor al, door bijvoorbeeld te anticiperen op een levensfase waarin belemmeringen niet ongebruikelijk meer zijn. Het gebruik van ‘<span class=\"italic _p_nadrukcur\">eigen kracht’</span> betekent ook dat de cliënt zelf voorziet in de kosten of voorziening die algemeen gebruikelijk zijn. Gebruik maken van de ‘<span class=\"italic _p_nadrukcur\">eigen kracht’</span> veronderstelt daarnaast dat de cliënt zich voldoende verzekert, bijvoorbeeld door een passende aanvullende zorgverzekering af te sluiten die aansluit bij de situatie van de cliënt. Om een goede beoordeling van de ‘eigen kracht’ te kunnen maken is het gewenst dat het gesprek niet alleen met de cliënt, maar ook met leden uit zijn sociaal netwerk en eventuele hulpverlener wordt gevoerd. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-71\"><!----></a><span class=\"italic _p_nadrukcur\">Inzet sociaal netwerk</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-72\"><!----></a>De inzet van het ‘<span class=\"italic _p_nadrukcur\">sociale netwerk’</span> wordt in de Wmo 2015 als volgt omschreven:<span class=\"italic _p_nadrukcur\">Personen uit de huiselijke kring of andere personen met wie de cliënt een sociale relatie onderhoudt.</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-73\"><!----></a>Ook bij het beoordelen van de hulpvraag voor Beschermd Wonen wordt altijd gekeken of door inzet van het ‘<span class=\"italic _p_nadrukcur\">sociale netwerk’</span> een plek in een Beschermde Woonvoorziening kan worden voorkomen.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-74\"><!----></a>Voor zover het gebruikelijk is dat partners, ouders, inwonende kinderen of andere huisgenoten elkaar bepaalde ondersteuning bieden, is een cliënt niet aangewezen op ondersteuning in de vorm van ondersteuning vanuit de gemeente. Ook wordt gekeken wat vanuit het bredere sociale netwerk aan ondersteuning kan worden geboden. Hiermee wordt onder andere uitwonende familieleden of bekenden bedoeld.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-75\"><!----></a>Er wordt geen maatwerkvoorziening verstrekt voor zover de cliënt met gebruikelijke hulp, met mantelzorg of met hulp van andere personen uit zijn sociale netwerk de beperkingen kan wegnemen. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-76\"><!----></a><span class=\"italic _p_nadrukcur\">Algmeen gebruikelijke voorzieningen</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-77\"><!----></a> ‘Algemeen gebruikelijke voorzieningen’ wordt in de wet niet nader gedefinieerd.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-78\"><!----></a>‘Algemeen gebruikelijk’ slaat enerzijds op voorzieningen en anderzijds op kosten. Een ‘algemeen gebruikelijke voorziening’ is een voorziening die niet bij wet, waaronder de Wmo 2015, wordt aangeboden en die, indien voorhanden, in redelijkheid een oplossing kan bieden voor de ondersteuningsbehoefte van de cliënt. Het gaat hierbij om voorzieningen zoals een boodschappendienst, een maaltijdvoorziening, gemaksdiensten, hondenuitlaatdienst, klussendienst, niet wettelijke kinderopvang, voorzieningen die via een aanvullende zorgverzekering wordt aangeboden, alarmering etc.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-79\"><!----></a>Daarnaast kan het gaan om voorzieningen die:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-80-1-2\"><!----></a>Niet speciaal bedoeld zijn voor mensen met een beperking;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-80-2-2\"><!----></a>In de reguliere handel verkrijgbaar zijn;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-80-3-2\"><!----></a>Niet meer kosten dan een vergelijkbaar product.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-81\"><!----></a>Daarnaast zijn er kosten waarmee een cliënt te maken krijgt, die ‘algemeen gebruikelijk’ zijn en niet specifiek zijn voor iemand met een beperking. Het kan bijvoorbeeld gaan om de kosten die samenhangen met het gebruik van een <span class=\"italic _p_nadrukcur\">‘algemeen gebruikelijke voorziening’</span> of algemene voorziening of de kosten van een verhuizing.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-82\"><!----></a>Er zal in relatie tot de cliënt bekeken worden of de voorziening of de kosten ‘algemeen gebruikelijk’ zijn. Het is in principe de cliënt die moet aantonen dat een ‘algemeen gebruikelijke voorziening’ voor hem niet tot de – financiële – mogelijkheden behoort. Daarbij kan het inkomen van de cliënt een rol spelen, maar ook het feit dat hij door een schuldsaneringtraject of beslag op zijn inkomen geen financiële ruimte meer heeft om te sparen of een lening af te sluiten. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-83\"><!----></a>Als er sprake is van een plotselinge optredende, onvoorziene noodzaak kunnen voorzieningen of kosten, die normaal gesproken als ‘algemeen gebruikelijk’ worden aangemerkt, dat toch niet zijn. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-84\"><!----></a><span class=\"italic _p_nadrukcur\">Algemene voorziening</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-85\"><!----></a>Een ‘algemene voorziening’ is in de wet als volgt gedefinieerd:</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-86\"><!----></a>Aanbod van diensten of activiteiten dat, zonder voorafgaand onderzoek naar de behoeften, persoonskenmerken en mogelijkheden van de gebruikers, toegankelijk is en dat is gericht op het versterken van zelfredzaamheid en participatie, of op opvang.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-87\"><!----></a>‘Algemene voorzieningen’ zijn voorzieningen die vrij toegankelijk zijn. Wel kunnen er globale restricties en toegangscriteria worden gesteld. Bijvoorbeeld aan de frequentie waarmee de voorziening wordt bezocht of dat men behoort tot de doelgroep waarvoor de voorziening is bedoeld. Aan deelname kunnen kosten verbonden zijn. Deze zijn voor rekening van de cliënt. Voorbeelden van algemene voorzieningen zijn het openbaar vervoer en het aanbod in wijk- en buurtcentra.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-88\"><!----></a>Vaststelling van de psychische aandoening of psychosociale beperking kan van belang zijn voor de afbakening van de Wmo 2015 en de Wet langdurige zorg (Wlz) en de Zorgverzekeringswet (Zvw). </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-89\"><!----></a>Een Beschermd Wonen pakket is een integraal pakket, waarvan hulp bij het huishouden een onderdeel is.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-90\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 9 Toegang in relatie tot regiobinding</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-91\"><!----></a>Bij Beschermd Wonen is sprake van landelijke toegankelijkheid. Dit betekent dat burgers zich in principe tot iedere gemeente kunnen wenden voor Beschermd Wonen. Het betekent niet dat in iedere gemeente een vorm van Beschermd Wonen geboden moet worden. Verder is een cliënt die vanuit Beschermd Wonen zelfstandig gaat wonen vrij zijn woonplaats te kiezen.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-92\"><!----></a>Om te voorkomen dat er een onevenredige spreiding van Beschermde Woonplekken over het land optreedt, zijn landelijke afspraken over de toegang noodzakelijk. Meerdere centrumgemeenten in de provincie Gelderland en Overijssel hebben vooruitlopend hierop al onderlinge afspraken gemaakt. De gemeente Doetinchem sluit zich als centrumgemeente hierbij aan.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-93\"><!----></a>De centrumgemeente gaat ervan uit dat alle cliënten primair in de eigen woonregio beschermd kunnen wonen. Dat betekent dan ook dat een plek voor Beschermd Wonen bij voorkeur wordt verstrekt door de centrumgemeente waar de voorwaarden voor herstel, zelfredzaamheid en participatie voor de cliënt het gunstigst zijn. Dit noemen we ’regiobinding. In dit licht zijn de toelatingscriteria bedoeld. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-94\"><!----></a>Vooruitlopend op de landelijke afspraken over de toegang Beschermd Wonen hanteert de centrumgemeente daarom de volgende uitgangspunten:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-95-1-2\"><!----></a>Alle cliënten moeten primair in de eigen regio beschermd kunnen wonen. Daarbij wordt er altijd eerst gezocht naar een passende plek binnen de eigen regio;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-95-2-2\"><!----></a>In het geval dat de cliënt zelf de uitdrukkelijke wens heeft om in een andere regio te gaan wonen, dan wordt daar vanuit de landelijke toegankelijkheid zoveel mogelijk aan meegewerkt. De centrumgemeente waar de cliënt woont, <span class=\"italic _p_nadrukcur\">‘de woonregio’</span>, neemt daartoe contact op met de centrumgemeente van voorkeur <span class=\"italic _p_nadrukcur\">‘wensregio’</span>. Dan wordt afgesproken wie het onderzoek doet: bij voorkeur is dat de wensregio, aangezien deze uiteindelijk de beschermde woonplek dient te bekostigen. In het geval de wensregio het onderzoek door de woonregio wil laten verrichten, zal de wensregio de uitkomst in beginsel overnemen. Bij verschillen in toelatingscriteria, wanneer heeft iemand welke vorm van Beschermd Wonen nodig, wordt gepoogd overeenstemming te bereiken. De wensregio verleent vervolgens toegang of biedt een alternatief en draagt zorg voor financiering;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-95-3-2\"><!----></a>In het geval er in de woonregio geen plek is of de kansen op een passend en succesvol traject in een andere regio nadrukkelijk beter zijn, neemt de consulent Beschermd Wonen van de centrumgemeente van de woonregio contact op met de consulent Beschermd Wonen van de andere centrumgemeente van de wensregio. Zij bespreken de casus, de mogelijkheden en zorgen bij een overgang voor een warme overdracht. In het geval dat een tijdelijke plaatsing wordt afgesproken, wordt deze plaats gefinancierd vanuit de woonregio. Bij definitieve plaatsing verleent de wensregio toegang en draagt zorg voor de financiering. Onderdeel van het gesprek tussen de medewerkers van de toegang is dat er wordt onderbouwd waarom er geen passende plek binnen de woonregio beschikbaar is, dan wel waarom een plek in de wensregio zorginhoudelijk beter voor de cliënt is. Als inhoudelijk handvat hanteert de centrumgemeente van de woonregio de handreiking voor landelijke toegankelijkheid in de maatschappelijke opvang.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-95-4-2\"><!----></a>Deze afspraken vervallen automatisch wanneer landelijke afspraken met betrekking tot de landelijke toegang in werking treden.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-96\"><!----></a>Is er regiobinding met de regio Achterhoek dan kan de cliënt beschermd wonen in de regio Achterhoek. Wanneer er in de betreffende regiogemeente geen beschermde woonvormen zijn, wordt in overleg met de cliënt Beschermd Wonen aangeboden in een andere Achterhoekse gemeente, waar plaats is. </p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-97\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 10 Woonverblijf in een sectorvreemde instelling</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-98\"><!----></a>In een aantal gevallen komt het in de regio Achterhoek voor dat cliënten met GGZ-C indicatie verblijven in een instelling, die niet overeenkomt met de formele toelating van de instelling waar de zorg wordt ontvangen. In veel gevallen blijkt dat deze cliënten binnen die instelling een goede beschermde woonplek hebben. De centrumgemeente vindt het belangrijk dat cliënten Beschermd Wonen binnen de meest passende instelling gehuisvest kunnen worden. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-99\"><!----></a>De centrumgemeente maakt hierover afspraken met de zorgaanbieders in hoeverre een instelling in staat is om passende ondersteuning te leveren. Uitgangspunt is dat een zorginstelling moet zijn toegerust om ondersteuning te kunnen bieden aan de GGZ-C cliëntgroep. Dit betekent dat de instelling moet beschikken over woonruimte met voldoende benodigde voorzieningen en personeel dat deskundig en vaardig is.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-100\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 11 Toekenning, resultaat, omvang en duur</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-101\"><!----></a><span class=\"italic _p_nadrukcur\">Beschikking</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-102\"><!----></a>De beschikking wordt afgegeven met daarin benoemd de meest passende plek. In de beschikking dient duidelijk te worden aangegeven dat het besluit iemand Beschermd Wonen toe te kennen, niet het recht geeft op een plaats in een bepaalde instelling voor Beschermd Wonen. Dit is afhankelijk van het beschikbaar zijn van een passende plek.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-103\"><!----></a>Het gaat bij Beschermd Wonen om het bieden van onderdak en begeleiding aan cliënten met psychische of psychosociale problemen. Een beschermde woonvorm biedt geen behandeling. Als een cliënt behandeling nodig heeft, dient hij dit zelf te regelen, eventueel met behulp van de begeleider.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-104\"><!----></a><span class=\"italic _p_nadrukcur\">Resultaat</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-105\"><!----></a>Het resultaat van het beschermd wonen is het realiseren van een situatie waarin de cliënt in staat wordt gesteld zich zo snel mogelijk weer op eigen kracht te handhaven of indien dit niet mogelijk is, zich met een toenemende mate van zelfredzaamheid in de samenleving te handhaven. Daarbij wordt onderscheid gemaakt tussen twee hoofddoelen:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>a.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-106-1-2\"><!----></a>herstel en uitstroom;</p></div></li><li class=\"li _p_li\"><span>b.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-106-2-2\"><!----></a>herstel en stabilisatie.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-107\"><!----></a>Onder deze twee hoofddoelen vallen verschillende subdoelen:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-108-1-2\"><!----></a>Het bevorderen en herstel van zelfredzaamheid en participatie;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-108-2-2\"><!----></a>Het bevorderen van het psychisch en psychosociaal functioneren;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-108-3-2\"><!----></a>Stabilisatie van een psychiatrisch ziektebeeld;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-108-4-2\"><!----></a>Het bieden van een veilige woonomgeving;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-108-5-2\"><!----></a>Het voorkomen van verwaarlozing, maatschappelijke overlast en het afwenden van gevaar voor de cliënt of voor de anderen.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-109\"><!----></a><span class=\"italic _p_nadrukcur\">Omvang</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-110\"><!----></a>De centrumgemeente hanteert voor 2016 deze zes zorgzwaartepakketten. Anders dan in de Algemene Wet Bijzondere Ziektekosten (AWBZ) kunnen de pakketten 1C GGZ en 2C GGZ zonder restricties van doelgroepen worden toegekend. De overweging voor een keuze van een pakket vindt plaats op basis van het best passend cliëntprofiel. Voor de cliëntprofielen wordt verwezen naar bijlage 1.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-111\"><!----></a>Voor na 2016 vindt mogelijk een andere indeling plaats.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-112\"><!----></a><span class=\"italic _p_nadrukcur\">Duur toekenning</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-113\"><!----></a>De decentralisatie van Beschermd Wonen, GGZ-cliëntgroep, heeft onder andere tot doel mensen niet langer dan nodig institutioneel en beschermd te laten wonen. Door het stimuleren van de eigen kracht en het uitgaan van de eigen mogelijkheden van de cliënt kan de huidige gemiddelde verblijfsduur mogelijk worden bekort. Het stellen van een geldigheidsduur van de toekenning op Beschermd Wonen voor maximaal twee jaar ligt in de rede. Dit om sturing te geven aan het structureel begeleiden van mensen in Beschermd Wonen en daar waar het kan mensen daadwerkelijk de kans te geven door te stromen. Daarnaast is een geldigheidsduur die te overzien is van belang in verband met de Wlz die op termijn toegankelijk wordt voor de ZZP GGZ C-groep.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-114\"><!----></a>Indien blijkt dat na deze periode een maatwerkvoorziening nodig blijft, vindt opnieuw een beoordeling plaats.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-115\"><!----></a>Landelijke objectieve inhoudelijke criteria voor de Wlz komen naar verwachting beschikbaar in 2017.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-116\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 12 Tijdelijk verblijf elders</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-117\"><!----></a>De cliënt kan pas weer instromen in de Beschermde Wooninstelling als er plaats is. Ook kan de cliënt in overleg met de consulent Beschermd Wonen ervoor kiezen zijn ondersteuning bij een andere gecontracteerde aanbieder voor Beschermd Wonen te verzilveren. De indicatie blijft geldig tot de einddatum die is opgenomen in de beschikking.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-118\"><!----></a>Vervolgens zal door de consulent Beschermd Wonen worden bepaald of een maatwerkoplossing noodzakelijk is.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-119\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 13 Overbruggingszorg</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-120\"><!----></a>In het geval dat een cliënt op de wachtlijst staat voor verblijf in een beschermde wooninstelling wordt de overbruggingszorg door de centrumgemeente bekostigd. De overbruggingszorg bestaat uit de professionele ondersteuning die totdat de cliënt wordt opgenomen, ambulant wordt geboden.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-121\"><!----></a>Wanneer een indicatie voor de Wet langdurige zorg is toegekend, komt de bekostiging van de overbruggingszorg ten laste van de Wet langdurige zorg.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-122\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 14 Wat niet wordt vergoed vanuit een persoonsgebonden budget</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-123\"><!----></a>Er wordt geen toeslag op een persoonsgebonden budget toegekend voor kleinschalige wooninitiatieven. In het geval dat een client overgangsrecht heeft, geldt dit ook voor de toeslag kleinschalige wooninitiatieven.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-124\"><!----></a>Er wordt geen eenmalige uitkering aan de zorgverlener toegekend bij beëindiging van de zorgovereenkomst uit een persoonsgebonden budget van rechtswege zoals door overlijden van de cliënt. In het geval van een persoonsgebonden budget heeft de gemeente geen directe relatie met de zorgverlener. De zorgverlener is met de cliënt een overeenkomst aangegaan en daarmee heeft de gemeente geen verantwoordelijkheid jegens de zorgverlener.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-125\"><!----></a>De centrumgemeente heeft geen inkooprelatie met de pgb-zorgaanbieder. De zorgaanbieder kan bij de centrumgemeente alleen kosten in rekening brengen, die voortkomen uit de daadwerkelijk geleverde ondersteuning die staat beschreven in de beschikking.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-126\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 15 Wijze van verstrekking</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-127\"><!----></a><span class=\"italic _p_nadrukcur\">De voorziening in natura</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-128\"><!----></a>De eerste mogelijkheid is de voorziening in natura. Het verkrijgen van een voorziening in natura gaat via een van de gecontracteerde aanbieders, waarmee de centrumgemeente afspraken heeft gemaakt. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-129\"><!----></a><span class=\"italic _p_nadrukcur\">Het persoonsgebonden budget</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-130\"><!----></a>De tweede mogelijkheid is de in de Wet maatschappelijke ondersteuning 2015 verplicht gestelde mogelijkheid een alternatief te bieden in de vorm van een persoonsgebonden budget.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-131\"><!----></a>In de Memorie van Toelichting op de wet wordt een drietal voorwaarden gegeven, waaraan voldaan moet zijn;</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>1.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-132-1-2\"><!----></a>De Wmo-consulent acht de cliënt of diens vertegenwoordiger in staat de aan het persoonsgebonden budget verbonden taken zoals het sluiten van overeenkomsten en het aansturen en aanspreken van de hulpverlener op zijn verplichtingen op een verantwoorde wijze uit te voeren.</p></div></li><li class=\"li _p_li\"><span>2.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-132-2-2\"><!----></a>De aanvrager stelt zich gemotiveerd op het standpunt dat het door de gemeente gecontracteerde aanbod niet passend is voor zijn specifieke situatie zoals niet goed vooraf planbare hulp of ondersteuning, op ongebruikelijke tijden of op veel korte momenten gedurende de dag.</p></div></li><li class=\"li _p_li\"><span>3.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-132-3-2\"><!----></a>Diensten, hulpmiddelen, woningaanpassingen en andere maatregelen moeten van goede kwaliteit zijn.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-133\"><!----></a>Naast bovengenoemde wettelijke voorwaarden stelt de centrumgemeente de volgende aanvullende voorwaarden:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>4.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-134-1-2\"><!----></a>Het persoonsgebonden budget kan uitsluitend worden ingezet voor kosten van de toegekende maatwerkvoorziening;</p></div></li><li class=\"li _p_li\"><span>5.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-134-2-2\"><!----></a>Een voorziening die is aangeschaft met een persoonsgebonden budget kan, zodra deze voorziening niet meer wordt gebruikt, onder verrekening van eventueel ingebrachte eigen middelen, door de gemeente worden opgehaald en voor herverstrekking beschikbaar worden gesteld;</p></div></li><li class=\"li _p_li\"><span>6.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-134-3-2\"><!----></a>Op grond van artikel 6.1 lid 4. van de Verordening is het toegestaan dat een cliënt met het persoonsgebonden budget diensten, hulpmiddelen, woningaanpassingen en andere maatregelen betrekt van een persoon die behoort tot het sociale netwerk, mits:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-134-3-3-1-2\"><!----></a>De ondersteuning aan de cliënt niet leidt tot overbelasting bij de persoon die deze niet professionele hulp of ondersteuning biedt;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-134-3-3-2-2\"><!----></a>De persoon die de ondersteuning biedt op geen enkele wijze druk heeft uitgeoefend op de cliënt.</p></div></li></ul></div></li></ul><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>7.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-135-1-2\"><!----></a>De cliënt draagt de verantwoordelijkheid voor het bewaken van de kwaliteit en continuïteit van de ondersteuning die hij betrekt van personen die tot het sociale netwerk behoren.</p></div></li><li class=\"li _p_li\"><span>8.</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-135-2-2\"><!----></a>De betrokken professionele hulpverlener kan geen budgethouder voor de cliënt zijn. </p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-136\"><!----></a>In het geval dat verwacht wordt dat het persoonsgebonden budget niet zal worden besteed waarvoor het is bedoeld, zal de consulent Beschermd Wonen het persoonsgebonden budget weigeren.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-137\"><!----></a><span class=\"italic _p_nadrukcur\">Het zorg- en budgetplan</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-138\"><!----></a>Bij de (her)beoordeling van de indicatie wordt het zorg- en budgetplan geëvalueerd. Het zorg- en budgetplan moet duidelijk maken dat de zorgaanbieder kwalitatief goede zorg levert. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-139\"><!----></a>In het zorg- en budgetplan staat, onder meer:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-1-2\"><!----></a>Korte omschrijving cliënt (zoals leeftijd, school, werk, hobby’s, diagnose/problematiek);</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-2-2\"><!----></a>Aan welke doelen tijdens de vorige indicatieperiode (indien sprake is van een herindicatie) is gewerkt;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-3-2\"><!----></a>Welke resultaten zijn behaald;</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>o</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-3-3-1-2\"><!----></a>Progressie/ontwikkeling;</p></div></li><li class=\"li _p_li\"><span>o</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-3-3-2-2\"><!----></a>Stabilisatie;</p></div></li><li class=\"li _p_li\"><span>o</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-3-3-3-2\"><!----></a>Terugval of escalatie</p></div></li></ul></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-4-2\"><!----></a>Met welk gedrag de komende periode wordt geoefend;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-5-2\"><!----></a>Hoe dit gedrag wordt geoefend;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-6-2\"><!----></a>Hoeveel uur per week de cliënt of de zorgverlener aan het werken aan deze doelen besteedt;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-7-2\"><!----></a>Na hoeveel maanden de doelen zijn bereikt;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-8-2\"><!----></a>Welke blijvende ondersteuning wordt geboden.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-9-2\"><!----></a>op welke manier deze ondersteuning bijdraagt aan zijn participatie en zelfredzaamheid;</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-10-2\"><!----></a>hoe de veiligheid, doeltreffendheid en cliëntgerichtheid van de ondersteuning is gewaarborgd; </p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-11-2\"><!----></a>hoe de continuïteit van de ondersteuning bij ziekte of andere uitval wordt gegarandeerd; </p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-140-12-2\"><!----></a>bij ondersteuning uit het sociaal netwerk motiveert de zorgverlener in het zorgplan waarom de zorgverlener capabel is om de ondersteuning te bieden.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-141\"><!----></a>Zowel de bedragen voor zorg in natura als de hoogte van het persoonsgebonden budget zijn vastgelegd in het Besluit maatschappelijke ondersteuning Montferland 2015. Jaarlijks kunnen de bedragen wijzigen.</p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-142\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 16 Niet verzilvering</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-143\"><!----></a>Er zijn situaties denkbaar zoals wachtlijsten, opname van cliënt in het ziekenhuis waardoor, volledige, verzilvering binnen de gehanteerde periode van zes maanden niet kan plaatsvinden.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-144\"><!----></a>De consulent Beschermd Wonen beoordeelt in overleg met de client en/of zijn vertegenwoordiger of hulpverlener in dat geval of een verlenging of aanpassing van de maatwerkoplossing noodzakelijk is. </p><p class=\"tussenkop tussenkop_cur _p_tussenkopcur\"><a name=\"id1-3-2-4-145\"><!----></a><span class=\"vet _p_nadrukvet\">Artikel 17 Voorkoming oneigenlijk gebruik en misbruik</span></p><p class=\"al _p_al\"><a name=\"id1-3-2-4-146\"><!----></a>In de Verordening maatschappelijke ondersteuning gemeente Doetinchem 2015 is ter voorkoming van oneigenlijk gebruik van een verstrekte voorziening de mogelijkheid geboden om, al dan niet steekproefsgewijs, controle hierop uit te oefenen. Op grond van de Verordening is het college bevoegd nadere regels met betrekking tot de controle op de besteding te stellen.</p><p class=\"al _p_al\"><a name=\"id1-3-2-4-147\"><!----></a>Mocht er sprake zijn van een ten onrechte toegekende maatwerkvoorziening, dan vergemakkelijkt een duidelijke formulering in de beschikking een eventuele beëindiging of terugvordering van de maatwerkvoorziening. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-148\"><!----></a>In de Verordening is in artikel 15 de bepaling opgenomen dat, indien er, naar later blijkt, ten onrechte een maatwerkvoorziening is verstrekt, het college bevoegd is de maatwerkvoorziening geheel of gedeeltelijk terug te vorderen. </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-149\"><!----></a>Ook de Memorie van Toelichting op de wet geeft een aantal maatregelen aan ter voorkoming van misbruik of fraude. De regering hecht eraan dat gemeenten actief invulling geven aan de bestrijding van misbruik. </p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-150-1-2\"><!----></a>Allereerst dient een aanvrager of cliënt zich altijd zich te identificeren, zowel richting de gemeente als richting de aanbieder.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-150-2-2\"><!----></a>Een tweede maatregel, die door de wetgever is genomen, is de invoering van het trekkingsrecht via de Sociale Verzekeringsbank. Voor het trekkingsrecht via de Sociale Verzekeringsbank is gekozen om fraude met besteding van het persoonsgebonden budget te voorkomen.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-150-3-2\"><!----></a>Daarnaast zijn alle actoren die een rol spelen bij de uitvoering van deze wet zoals het CAK, de Sociale Verzekeringsbank, toezichthoudende ambtenaren, desgevraagd de persoonsgegevens, waaronder persoonsgegevens betreffende de gezondheid, aan de consulent Beschermd Wonen te verstrekken, voor zover de consulent Beschermd Wonen die gegevens nodig heeft voor de uitvoering van taken op grond van de Wet maatschappelijke ondersteuning 2015.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-151\"><!----></a>Het college hanteert met het oog de fraudepreventie de principes:</p><ul class=\"_explicit whitespace-small _p_lijst\"><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-152-1-2\"><!----></a>Vroegtijdig informeren: hierbij is een belangrijke rol weggelegd voor de consulent Beschermd Wonen. Hij informeert de cliënt vroegtijdig over hun rechten en plichten.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-152-2-2\"><!----></a>De cliënt meldt alle signalen, waarbij door zorgaanbieders andere ondersteuning wordt geboden, dan is overeengekomen.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-152-3-2\"><!----></a>Vroegtijdig detecteren en afhandelen: de consulent Beschermd Wonen is alert op fraudesignalen. </p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-152-4-2\"><!----></a>Buurtcoaches, wijkconsulenten en Stichting MEE Oost, melden fraudesignalen aan de consulent Beschermd Wonen.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-152-5-2\"><!----></a>Optimaliseren van de dienstverlening: bij de inrichting van de werkprocessen kijkt de centrumgemeente ook naar het effect van de werkprocessen op de bereidheid van cliënten om de regels na te leven.</p></div></li><li class=\"li _p_li\"><span>•</span><div><p class=\"al _p_al\"><a name=\"id1-3-2-4-152-6-2\"><!----></a>Daadwerkelijk sanctioneren: de centrumgemeente hanteert een sanctie- en opsporingsbeleid.</p></div></li></ul><p class=\"al _p_al\"><a name=\"id1-3-2-4-153\"><!----></a>  </p><p class=\"al _p_al\"><a name=\"id1-3-2-4-154\"><!----></a>    </p></div><div class=\"_p_bijlage\"><a name=\"id1-3-2-5\"><!----></a><a name=\"id1-3-2-5-1\"><!----></a><p class=\"hoofdstuk_kop _p_hoofdstuk_kop\"><a name=\"id1-3-2-5-1-3\"><!----></a><span class=\"_p_label\">Bijlage</span> <span class=\"_p_nr\">1</span> Cliëntprofielen</p><p class=\"al _p_al\"><a name=\"id1-3-2-5-2\"><!----></a>Per cliëntprofiel kan dagbesteding en vervoer geïndiceerd worden</p><div class=\"_p_table\"><a name=\"id1-3-2-5-3\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-1-1-1\"><!----></a>ZZP 1C GGZ Beschermd wonen met begeleiding</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-2-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Cliëntprofiel</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-2-1-2\"><!----></a>Deze cliëntgroep heeft vanwege een lichte psychiatrische aandoening, begeleiding en vooral bescherming en stabiliteit nodig, in een veilige en weinig eisende woonomgeving. De cliënten hebben ten aanzien van hun sociale redzaamheid in het algemeen dagelijks begeleiding nodig, die naar intensiteit beperkt kan zijn. Er is sprake van enig verlies van zelfregie en mogelijk van een verstoord dag- en nachtritme. Cliënten hebben in beperkte mate problemen met het onderhouden van sociale relaties, het deelnemen aan het maatschappelijk leven en het invullen van de dag. De cliënten beschikken vaak wel over vaardigheden om, zo nodig met enige ondersteuning, een (gezamenlijke) huishouding te voeren. De cliënten hebben in het algemeen ondersteuning nodig ten aanzien van de cognitieve/psychische functies. Dit speelt met name bij concentratie, geheugen en denken, motivatie en het psychosociaal welbevinden. De aard van het begeleidingsdoel is stabilisatie en continuering van de situatie, dan wel ontwikkelingsgericht. Ten aanzien van ADL hebben de cliënten in het algemeen enige behoefte aan toezicht of stimulatie met betrekking tot de persoonlijke verzorging, de intensiteit daarvan is beperkt. Ten aanzien van mobiliteit hebben de cliënten in het algemeen geen hulp nodig. Bij deze cliënten is in het algemeen geen sprake van gedragsproblematiek. De psychiatrische problematiek is bij deze cliënten in het algemeen passief van aard (er is geen floride psychopathologie). De psychiatrische symptomen zijn zodanig onder controle dat deze in het dagelijks leven geen overheersende rol spelen. Er is sprake van een goede reactie op (depot)medicatie. De zorgverlening is volgens afspraak en direct oproepbaar te leveren. De dominante grondslag voor dit cliëntprofiel is meestal een psychiatrische aandoening, psychische stoornis (waartoe ook verslaving behoort).</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-3-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Verblijfskenmerken</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-3-1-2\"><!----></a>Setting: beschut/beschermd wonen (geclusterd of groepswonen, dichtbij of in ‘moederhuis’), kleinschalige beschermende woonvormen, dependances.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-3-1-3\"><!----></a>Nachtdienst: wacht op afroep/slaapwacht.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-3-1-2-3-1-4\"><!----></a>Leveringsvoorwaarde: volgens afspraak en direct oproepbaar.</p></td></tr></tbody></table></div><p class=\"al _p_al\"><a name=\"id1-3-2-5-4\"><!----></a>   </p><div class=\"_p_table\"><a name=\"id1-3-2-5-5\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-1-1-1\"><!----></a>ZZP 2C GGZ Gestructureerd beschermd wonen met uitgebreide begeleiding</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Cliëntprofiel</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-2\"><!----></a>Deze cliëntgroep heeft vanwege een psychiatrische aandoening continu begeleiding nodig. De cliënten hebben een structuur, stabiliteit, bescherming en veiligheid biedende woonomgeving nodig waarin toezicht wordt gehouden en die weinig eisen stelt.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-3\"><!----></a>De cliënten hebben ten aanzien van hun sociale redzaamheid dagelijkse uitgebreide begeleiding nodig. Er is sprake van verlies van zelfregie en van een verstoord dagen nachtritme. Cliënten hebben problemen met het onderhouden van sociale relaties, het deelnemen aan het maatschappelijk leven en het invullen van de dag.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-4\"><!----></a>Daarnaast zijn er beperkingen in de besluitnemings- en oplossingsvaardigheden en bij het initiëren en uitvoeren van complexere taken. De cliënten hebben in het algemeen uitgebreide ondersteuning nodig ten aanzien van alle cognitieve/psychische functies.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-5\"><!----></a>De aard van het begeleidingsdoel is stabilisatie en continuering van de situatie dan wel ontwikkelingsgericht. </p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-6\"><!----></a>Ten aanzien van ADL hebben de cliënten in het algemeen enige behoefte aan toezicht of stimulatie met betrekking tot de persoonlijke verzorging, de intensiteit daarvan is beperkt.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-7\"><!----></a>Ten aanzien van mobiliteit hebben de cliënten in het algemeen geen hulp nodig. Bij deze cliënten kan terugkerend sprake zijn van enige gedragsproblematiek, maar die is hanteerbaar in de context van voortdurende begeleiding. Dit speelt met name bij manipulatief gedrag en reactief gedrag met betrekking tot interactie.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-8\"><!----></a>De psychiatrische problematiek is bij deze cliënten in het algemeen passief van aard (er is geen floride psychopathologie). De psychiatrische symptomen zijn zodanig onder controle dat deze in het dagelijks leven geen overheersende rol spelen. Er is sprake van een goede reactie op (depot)medicatie.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-2-1-9\"><!----></a>De zorgverlening is voortdurend in de nabijheid te leveren. De dominante grondslag voor dit cliëntprofiel is psychiatrische aandoening, psychische stoornis (waartoe ook verslaving behoort).</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-3-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Verblijfskenmerken</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-3-1-2\"><!----></a>Setting: beschut/beschermd wonen.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-3-1-3\"><!----></a>Nachtdienst: wakende wacht/in nabijheid.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-5-1-2-3-1-4\"><!----></a>Leveringsvoorwaarde: voortdurend in de nabijheid.</p></td></tr></tbody></table></div><p class=\"al _p_al\"><a name=\"id1-3-2-5-6\"><!----></a>   </p><div class=\"_p_table\"><a name=\"id1-3-2-5-7\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-1-1-1\"><!----></a>ZZP 3C GGZ Beschermd wonen met intensieve begeleiding</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-2-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Cliëntprofiel</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-2-1-2\"><!----></a>Deze cliëntgroep heeft vanwege een psychiatrische aandoening intensieve begeleiding nodig. De cliënten hebben een veilige, weinig eisende en prikkelarme woonomgeving nodig die bescherming, stabiliteit en structuur biedt. De symptomatologie is bij deze cliënten naar de achtergrond geschoven en de ‘defecten’ staan op de voorgrond. De begeleiding is dan ook met name gericht op het omgaan met deze defecten. De cliënten hebben ten aanzien van hun sociale redzaamheid dagelijks intensieve begeleiding nodig. Er is sprake van verlies van zelfregie en van een verstoord dagen nachtritme. Cliënten hebben grote problemen met het onderhouden van sociale relaties en het invullen van de dag. Tot deelname aan het maatschappelijk leven is men nagenoeg niet in staat en vaak ook niet geïnteresseerd. Daarnaast zijn er forse beperkingen in de besluitnemings- en oplossingsvaardigheden en bij het initiëren en uitvoeren van eenvoudige en complexere taken. De cliënten hebben in het algemeen begeleiding nodig bij het beheren van geld en het verrichten van administratieve handelingen. Ze reizen doorgaans met begeleiding. De cliënten hebben in het algemeen intensieve ondersteuning nodig ten aanzien van alle cognitieve/psychische functies. De aard van het begeleidingsdoel is stabilisatie en continuering van de situatie dan wel ontwikkelingsgericht. Er kan echter ook sprake zijn van begeleiding bij achteruitgang. Ten aanzien van ADL hebben de cliënten in het algemeen behoefte aan toezicht of stimulatie met betrekking tot de persoonlijke verzorging. Ten aanzien van mobiliteit hebben de cliënten in het algemeen geen hulp nodig. Bij deze cliënten kan terugkerend sprake zijn van enige gedragsproblematiek, maar die is hanteerbaar in de context van voortdurende begeleiding. Dit speelt met name bij reactief gedrag met betrekking tot interactie. De psychiatrische problematiek bij deze cliënten varieert van passief tot actief. De psychiatrische symptomen zijn bij tijd en wijle lastig onder controle te krijgen; dan is intensivering van zorg gewenst (of bijstelling van medicatie). De zorgverlening is voortdurend in de nabijheid te leveren. De dominante grondslag voor dit cliëntprofiel is psychiatrische aandoening, psychische stoornis (waartoe ook verslaving behoort).</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-3-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Verblijfskenmerken</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-3-1-2\"><!----></a>Setting: </p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-3-1-3\"><!----></a>beschut/beschermd wonen</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-3-1-4\"><!----></a>Nachtdienst: wakende wacht/in nabijheid.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-7-1-2-3-1-5\"><!----></a>Leveringsvoorwaarde: voortdurend in de nabijheid.</p></td></tr></tbody></table></div><p class=\"al _p_al\"><a name=\"id1-3-2-5-8\"><!----></a>  </p><div class=\"_p_table\"><a name=\"id1-3-2-5-9\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-9-1-2-1-1-1\"><!----></a>ZZP 4C GGZ Gestructureerd beschermd wonen met intensieve begeleiding en verzorging</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-9-1-2-2-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Cliëntprofiel</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-9-1-2-2-1-2\"><!----></a>Deze cliëntgroep heeft vanwege een complexe psychiatrische aandoening intensieve begeleiding nodig. De cliënten hebben een structuur en toezicht biedende beschermende woonomgeving nodig, die deels een besloten karakter kan hebben (gecontroleerde in- en uitgang). Er is ondersteuning van taken op alle levensterreinen nodig inclusief hulp vanwege (somatische) gezondheidsbeperkingen. De cliënten hebben ten aanzien van hun sociale redzaamheid dagelijks intensieve begeleiding nodig. Er is sprake van verlies, van zelfregie en van een verstoord dagen nachtritme. Cliënten hebben grote problemen met het onderhouden van sociale relaties en het invullen van de dag. Tot deelname aan het maatschappelijk leven is men niet in staat. Daarnaast zijn er forse beperkingen in de besluitnemings- en oplossingsvaardigheden en bij het initiëren en uitvoeren van eenvoudige en complexere taken. De cliënten hebben in het algemeen begeleiding nodig bij het beheren van geld en het verrichten van administratieve handelingen. Ze reizen doorgaans met begeleiding. De cliënten hebben in het algemeen intensieve ondersteuning nodig ten aanzien van alle cognitieve/psychische functies. De aard van het begeleidingsdoel is stabilisatie en continuering van de situatie, dan wel ontwikkelingsgericht. Ten aanzien van ADL hebben de cliënten vanwege gezondheidsproblemen vaak dagelijks behoefte aan hulp bij de persoonlijke verzorging (bijvoorbeeld als gevolg van problemen passend bij het ouder worden of door verwaarlozing van de gezondheid door het zwerven op straat). Ten aanzien van mobiliteit hebben de cliënten in het algemeen geen hulp nodig. Bij deze cliënten is sprake van gedragsproblematiek. De begeleiding is mede gericht op het beheersbaar houden van deze gedragsproblematiek en het omgaan met defecten. De psychiatrische problematiek is bij deze cliënten actief van aard (de psychopathologie is floride, en/of er is sprake van actieve middelen verslaving). De problematiek wordt getracht onder controle te houden met medicijnen, indien van toepassing gecontroleerd gebruik van middelen en intensieve begeleiding. De zorgverlening is voortdurend in de nabijheid te leveren. De dominante grondslag voor dit cliëntprofiel is psychiatrische aandoening, psychische stoornis (waartoe ook verslaving behoort), mogelijk in combinatie met een somatische aandoening, een lichamelijke en/of verstandelijke handicap.</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-9-1-2-3-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Verblijfskenmerken</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-9-1-2-3-1-2\"><!----></a>Setting: beschut/beschermd groepswonen (eventueel besloten karakter).</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-9-1-2-3-1-3\"><!----></a>Nachtdienst: wakende wacht/in nabijheid. Leveringsvoorwaarde: voortdurend in de nabijheid.</p></td></tr></tbody></table></div><p class=\"al _p_al\"><a name=\"id1-3-2-5-10\"><!----></a>   </p><div class=\"_p_table\"><a name=\"id1-3-2-5-11\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-1-1-1\"><!----></a>ZZP 5C GGZ Beschermd wonen met intensieve begeleiding en gedragsregulering</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-2-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Cliëntprofiel</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-2-1-2\"><!----></a>Deze cliëntgroep heeft vanwege een complexe psychiatrische aandoening intensieve zorg en intensieve begeleiding nodig. De woonomgeving moet veel structuur, veiligheid en bescherming bieden, die deels een besloten karakter kan hebben (gecontroleerde in- en uitgang). Er is ondersteuning en overname van taken op alle levensterreinen nodig. De cliënten hebben ten aanzien van hun sociale redzaamheid dagelijks intensieve begeleiding nodig die voortdurend nabij is, met daarnaast een sterk gestructureerde dagindeling. Cliënten zijn nauwelijks in staat sociale relaties te onderhouden en de dag in te vullen. Tot deelname aan het maatschappelijk leven is men niet in staat, noch geïnteresseerd. Daarnaast ontbreken de besluitnemings- en oplossingsvaardigheden en moet het initiëren en uitvoeren van eenvoudige en complexere taken vaak worden overgenomen. De cliënten hebben begeleiding nodig bij het beheren van geld en het verrichten van administratieve handelingen. Ze reizen met begeleiding. De cliënten hebben intensieve ondersteuning nodig ten aanzien van alle cognitieve/psychische functies. De aard van het begeleidingsdoel is stabilisatie en continuering van de situatie dan wel ontwikkelingsgericht. Ten aanzien van ADL hebben de cliënten betreffende de verschillende aspecten behoefte aan toezicht en stimulatie en/of hulp. Er kunnen somatische problemen zijn die extra aandacht vragen als gevolg van zelfverwaarlozing. Ten aanzien van mobiliteit hebben de cliënten in het algemeen geen hulp nodig. Bij deze cliënten is sprake van ernstige gedragsproblematiek die, mede door intensieve begeleiding, voortdurend gereguleerd moet worden. Deze cliënten doen een groot beroep op hun sociale omgeving en zetten deze voortdurend onder druk met manipulatief gedrag. Ze zijn beperkt gevoelig voor correctie, hebben weinig inzicht in hun eigen aandeel bij interactieproblemen en een relatief beperkt leervermogen. Er is sprake van verbaal agressief gedrag, manipulatief, dwangmatig, destructief en reactief gedrag met betrekking tot interactie. Er kan sprake zijn van zelfverwondend of zelfbeschadigend gedrag. De psychiatrische problematiek is bij deze cliënten actief van aard (de psychopathologie is floride, en/of er is sprake van actieve middelenverslaving). De problematiek wordt getracht onder controle te houden met medicijnen, indien van toepassing gecontroleerd gebruik van middelen en intensieve begeleiding. De zorgverlening is voortdurend in de nabijheid te leveren. De dominante grondslag voor dit cliëntprofiel is psychiatrische aandoening, psychische stoornis (waartoe ook verslaving behoort), mogelijk in combinatie met een somatische aandoening, een lichamelijke en/of verstandelijke handicap.</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-3-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Verblijfskenmerken</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-3-1-2\"><!----></a>Setting: beschut/beschermd groepswonen (eventueel besloten karakter).</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-3-1-3\"><!----></a>Nachtdienst: wakende wacht/in nabijheid. </p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-11-1-2-3-1-4\"><!----></a>Leveringsvoorwaarde: voortdurend in de nabijheid.</p></td></tr></tbody></table></div><p class=\"al _p_al\"><a name=\"id1-3-2-5-12\"><!----></a>   </p><div class=\"_p_table\"><a name=\"id1-3-2-5-13\"><!----></a><table class=\"_p_tgroup\"><col class=\"_p_colspec\" /><tbody class=\"_p_tbody\"><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-1-1-1\"><!----></a>ZZP 6C GGZ Beschermd wonen met intensieve begeleiding en intensieve verpleging en verzorging</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-2-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Cliëntprofiel</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-2-1-2\"><!----></a>Deze cliëntgroep heeft vanwege een complexe psychiatrische aandoening, in combinatie met een somatische aandoening, lichamelijke handicap of verstandelijke beperking, intensieve begeleiding en zorg nodig. De woonomgeving moet veel structuur, veiligheid en bescherming bieden en zijn aangepast aan de beperkingen van de cliënten (b.v. rolstoelgebruik). Er is veelal overname van taken op alle levensterreinen nodig. De cliënten hebben ten aanzien van hun sociale redzaamheid dagelijks intensieve begeleiding nodig die voortdurend nabij is, met daarnaast een sterk gestructureerde dagindeling. Cliënten zijn nauwelijks in staat sociale relaties te onderhouden en de dag in te vullen. Tot deelname aan het maatschappelijk leven is men niet in staat, noch geïnteresseerd. Daarnaast ontbreken de besluitnemings- en oplossingsvaardigheden en moet het initiëren en uitvoeren van eenvoudige en complexere taken vaak worden overgenomen. De cliënten hebben begeleiding nodig bij het beheren van geld en het verrichten van administratieve handelingen. Ze reizen met begeleiding. De cliënten hebben intensieve ondersteuning nodig ten aanzien van alle cognitieve/psychische functies. De aard van het begeleidingsdoel is divers; zowel stabilisatie en continuering van de situatie, ontwikkelingsgericht en begeleiding bij achteruitgang zijn aan de orde. Met betrekking tot ADL is uitgebreide behoefte aan hulp, onder andere bij het eten en drinken en bij het zich wassen en kleden. Ten aanzien van mobiliteit is met betrekking tot het maken van transfers (in en uit bed; in en uit rolstoel) hulp nodig. Bij deze cliënten is sprake van gedragsproblematiek. De begeleiding is mede gericht op het beheersbaar houden van deze gedragsproblematiek. De psychiatrische problematiek is bij deze cliënten actief van aard (de psychopathologie is floride). De problematiek wordt getracht onder controle te houden met medicijnen en intensieve begeleiding. De zorgverlening is voortdurend in de nabijheid of 24 uur per dag direct te leveren. De dominante grondslag voor dit cliëntprofiel is psychiatrische aandoening, psychische stoornis (waartoe ook verslaving behoort), in combinatie met een somatische aandoening, een lichamelijke en/of verstandelijke handicap.</p></td></tr><tr class=\"_p_row\"><td class=\"_p_entry\" rowspan=\"1\" colspan=\"1\"><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-3-1-1\"><!----></a><span class=\"vet _p_nadrukvet\">Verblijfskenmerken</span></p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-3-1-2\"><!----></a>Setting: 24-uurs woonvoorzieningen.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-3-1-3\"><!----></a>Nachtdienst: wakende wacht/in nabijheid.</p><p class=\"_p_table_al\"><a name=\"id1-3-2-5-13-1-2-3-1-4\"><!----></a>Leveringsvoorwaarde: voortdurend in de nabijheid of 24 uur per dag direct aanwezig.</p></td></tr></tbody></table></div></div></div></div></div></div></div></div></body></html>"

In [7]:

from pathlib import Path
from haystack.components.converters import HTMLToDocument

converter = HTMLToDocument()
docs = converter.run(html)

docs[0].text

# from boilerpy3 import extractors

# extractor = extractors.ArticleExtractor()


# # From raw HTML
# content = extractor.get_content(html)
# print(content)

Beleidsregels Beschermd Wonen Montferland 2016
Citeertitel
Beleidsregels Beschermd Wonen Montferland 2016
Vastgesteld door
college van burgemeester en wethouders
Onderwerp
Opmerkingen met betrekking tot de regeling
Geen.
Regelgeving die op deze regeling is gebaseerd (gedelegeerde regelgeving)
Geen.
Overzicht van in de tekst verwerkte wijzigingen
Datum inwerkingtreding
Terugwerkende kracht tot en met
Datum uitwerkingtreding
Beleidsregels Beschermd Wonen Gemeente Montferland 2016
Het college van burgemeester en wethouders van Montferland
Overwegende dat het de bevoegdheid heeft om regels te stellen omtrent de te verstrekken maatwerkvoorzieningen in de vorm van zorg in natura en in de vorm van het persoonsgebonden budget in het kader van de Wet maatschappelijke ondersteuning 2015 ;
Gelet op de Verordening maatschappelijke ondersteuning Montferland 2015 , verder te noemen Verordening, en artikel 4:81 Algemene wet bestuursrecht ;
Besluit:
Vast te stellen het navolgende:
Beleidsregels Besche

In [16]:

from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title

# elements = partition_html(text=html)
elements = partition_html(text=html, max_characters=1500, new_after_n_chars=1000, overlap=64)
chunks = chunk_by_title(elements)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

Chunk 0:
Zoek regelingen op overheid.nl

Ziet u een fout in deze regeling? Meld het ons op regelgeving@overheid.nl!

Bergen (NH)

beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016

Wetstechnische informatie


Chunk 1:
Organisatie Bergen (NH) Organisatietype Gemeente Officiële naam regeling beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016 Citeertitel beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016 Vastgesteld door college van burgemeester en wethouders


Chunk 2:
Onderwerp maatschappelijke zorg en welzijn Eigen onderwerp


Chunk 3:
Opmerkingen met betrekking tot de regeling

Geen

Wettelijke grondslag(en) of bevoegdheid waarop de regeling is gebaseerd

Participatiewet, art. 35

Regelgeving die op deze regeling is gebaseerd (gedelegeerde regelgeving)

Geen


Chunk 4:
Datum inwerkingtreding Terugwerkende kracht tot en met Datum uitw

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "jinaai/reader-lm-0.5b"

device = "cuda:1" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "user", "content": html}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)

print(input_text)

inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=1024, temperature=0, do_sample=False, repetition_penalty=1.08)

print(tokenizer.decode(outputs[0]))


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [6]:
import torch
torch.cuda.empty_cache()
